# Premium Feature of Detailed 

In [3]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob

from textblob import TextBlob

def dynamic_detailed_summarize_premium(text, detail_level=1, context_type='general', important_keywords=None):
    """
    Premium summarization function with context sensitivity, user customization, and sentiment analysis.
    
    Parameters:
    - text (str): the original text to be summarized
    - detail_level (int): level of detail for the summary (1: less detail, 2: more detail)
    - context_type (str): type of the text (e.g., 'story', 'article', 'report')
    - important_keywords (list): list of important keywords or phrases to include in the summary
    
    Returns:
    - str: the premium summary
    """
    
    # Split the text into sentences
    sentences = text.split('. ')
    
    # Dynamically set the number of sentences based on the length of the input text
    total_sentences = len(sentences)
    num_of_sentences = max(1, total_sentences // 5)  # Using 1/5 of the total number of sentences for summary
    
    # Text Preprocessing (TF-IDF Vectorization)
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()
    
    # Compute Cosine Similarity Matrix
    cosine_matrix = cosine_similarity(vectors)
    
    # Rank Sentences using PageRank algorithm
    nx_graph = nx.from_numpy_array(cosine_matrix)
    scores = nx.pagerank(nx_graph)
    
    # Sort the Sentences and Select Top N
    ranked_sentences = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)
    
    # Context Sensitivity: Adjust strategy based on context_type
    if context_type == 'story':
        # For stories, try to include the introduction and conclusion
        intro_sentence = sentences[0]
        conclusion_sentence = sentences[-1]
    elif context_type == 'article':
        # For articles, try to include the main points or arguments
        intro_sentence = sentences[0]
        conclusion_sentence = sentences[-1]
    else:
        intro_sentence = conclusion_sentence = None

    # User Customization: Include important keywords if specified
    if important_keywords:
        important_sentences = [s for s in sentences if any(kw.lower() in s.lower() for kw in important_keywords)]
    else:
        important_sentences = []
    
    # Sentiment Analysis: Use TextBlob to analyze the sentiment of each sentence
    sentiment_scores = [TextBlob(s).sentiment.polarity for s in sentences]
    
    # Combine PageRank scores and sentiment scores for final ranking
    combined_scores = [(0.7 * scores[i] + 0.3 * abs(sentiment_scores[i]), s, i) for i, s in enumerate(sentences)]
    ranked_sentences = sorted(combined_scores, reverse=True)
    
    # Select Top N sentences based on the combined scores
    if detail_level == 1:
        selected_sentences = [ranked_sentences[i] for i in range(min(num_of_sentences, len(ranked_sentences)))]
    else:
        selected_sentences = [ranked_sentences[i] for i in range(min(num_of_sentences * 2, len(ranked_sentences)))]
    
    # Make sure to include the introductory and concluding sentences, and important sentences
    if intro_sentence and intro_sentence not in [s[1] for s in selected_sentences]:
        selected_sentences.append((0, intro_sentence, sentences.index(intro_sentence)))
    if conclusion_sentence and conclusion_sentence not in [s[1] for s in selected_sentences]:
        selected_sentences.append((0, conclusion_sentence, sentences.index(conclusion_sentence)))
    for s in important_sentences:
        if s not in [s[1] for s in selected_sentences]:
            selected_sentences.append((0, s, sentences.index(s)))
    
    # Sort the selected sentences back into their original order
    sorted_selected_sentences = sorted(selected_sentences, key=lambda x: x[2])
    
    # Construct the summary
    summary = '. '.join([sentence[1] for sentence in sorted_selected_sentences])
    
    return summary

# Test the new premium summarization function
test_text = '''
Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. 
One day, he received a mysterious letter inviting him on an adventure. 
Henry was intrigued but also scared. 
After a lot of contemplation, he decided to take the risk. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Henry returned to his town, a changed man, content and fulfilled. 
And he lived happily ever after.
'''

premium_summary = dynamic_detailed_summarize_premium(test_text, detail_level=2, context_type='story', important_keywords=['journey', 'treasure'])
print("Premium Summary:")
print(premium_summary)



Premium Summary:

Once upon a time, in a small town, lived a man named Henry. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
And he lived happily ever after.



In [4]:
from sklearn.feature_extraction import text

# Modified function to get important keywords from text
def extract_important_keywords_improved(text_content, top_n=5):
    """
    Extracts important keywords from the given text using TF-IDF and excluding stopwords.
    
    Parameters:
    - text_content (str): the original text
    - top_n (int): number of top keywords to return
    
    Returns:
    - list: a list of top_n important keywords
    """
    # Add more stopwords to the default list if needed
    my_stop_words = list(text.ENGLISH_STOP_WORDS)
    
    vectorizer = TfidfVectorizer(stop_words=my_stop_words)
    vectors = vectorizer.fit_transform([text_content]).toarray()
    feature_names = vectorizer.get_feature_names_out()
    
    top_keywords = sorted(list(zip(feature_names, vectors[0])), key=lambda x: x[1], reverse=True)[:top_n]
    return [keyword[0] for keyword in top_keywords]

# Test the improved function
important_keywords_improved = extract_important_keywords_improved(test_text)
important_keywords_improved


['henry', 'man', 'journey', 'lived', 'town']

In [5]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# Your existing functions like dynamic_detailed_summarize_premium and extract_important_keywords_improved should be here

# Modify the premium summarization function to handle different detail levels more effectively
def dynamic_detailed_summarize_premium_adjusted(text, detail_level=1, context_type='general', important_keywords=None):
    if important_keywords is None:
        important_keywords = extract_important_keywords_improved(text)

    total_sentences = len(sent_tokenize(text))
    
    # Calculate the number of sentences to include in the summary
    num_sentences = total_sentences // (detail_level + 1)
    print('NUM SENTENCE IS-:',num_sentences)
    return dynamic_detailed_summarize_premium(text, num_sentences, context_type, important_keywords)


# Test the adjusted function with different detail levels
test_text_2 = '''
Artificial intelligence is revolutionizing various sectors, from healthcare to transportation. 
In healthcare, AI algorithms assist doctors in diagnosing diseases and recommending treatments. 
In transportation, autonomous vehicles promise to make roads safer and reduce traffic congestion. 
However, the widespread adoption of AI also raises ethical and social concerns, such as job displacement and data privacy. 
Regulations are needed to ensure that AI is developed and deployed in a manner that is beneficial for all.
'''

premium_summary_adjusted_1 = dynamic_detailed_summarize_premium_adjusted(test_text, detail_level=1, context_type='article')
premium_summary_adjusted_2 = dynamic_detailed_summarize_premium_adjusted(test_text, detail_level=2, context_type='article')

print("Premium Summary with Detail Level 1:")
print(premium_summary_adjusted_1)
print("\nPremium Summary with Detail Level 2:")
print(premium_summary_adjusted_2)


NUM SENTENCE IS-: 4
NUM SENTENCE IS-: 3
Premium Summary with Detail Level 1:

Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. 
Henry was intrigued but also scared. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Henry returned to his town, a changed man, content and fulfilled. 
And he lived happily ever after.


Premium Summary with Detail Level 2:

Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. 
Henry was intrigued but also scared. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Henry returned to his town, a changed man, content and fulfilled. 
And he lived happily ever after.



In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_important_keywords(text, num_keywords=5):
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
    X = vectorizer.fit_transform([text])
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:num_keywords]
    return top_n.tolist()

# Test the function
test_text = """Once upon a time, in a small town, lived a man named Henry. He was an ordinary man with a monotonous life. One day, he received a mysterious letter. The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. Intrigued but also scared, Henry decided to take the risk and embark on the journey."""
important_keywords = extract_important_keywords(test_text)
print("Important Keywords:", important_keywords)


Important Keywords: ['man', 'henry', 'letter', 'lived', 'day']


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.feature_extraction import text as sk_text
import re

def dynamic_detailed_summarize_premium(text, detail_level=1, context_type='general', important_keywords=[]):
    # Preprocessing the text
    sentences = text.split('.')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]

    # Add custom stop words or use default English stop words
    my_stop_words = list(sk_text.ENGLISH_STOP_WORDS)

    # Vectorization
    vectorizer = CountVectorizer(stop_words=my_stop_words).fit_transform(sentences)
    vectors = vectorizer.toarray()

    # Cosine Similarity
    cosine_matrix = cosine_similarity(vectors)
    
    # Applying PageRank Algorithm
    nx_graph = nx.from_numpy_array(cosine_matrix)
    scores = nx.pagerank(nx_graph)

    # Sort sentences by score to get the summary
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Adding weight for important keywords
    for i in range(len(ranked_sentences)):
        for keyword in important_keywords:
            if keyword.lower() in ranked_sentences[i][1].lower():
                ranked_sentences[i] = (ranked_sentences[i][0] * 1.2, ranked_sentences[i][1])

    # Re-sort sentences after adding keyword weights
    ranked_sentences = sorted(ranked_sentences, key=lambda x: x[0], reverse=True)

    # Selecting sentences based on detail level and reordering them based on their appearance in the original text
    num_sentences = len(sentences)

    # Decide the number of sentences based on detail level
    if detail_level == 1:
        num_summary_sentences = max(3, int(num_sentences * 0.2))
    else:
        num_summary_sentences = max(5, int(num_sentences * 0.4))

    selected_sentences = [ranked_sentences[i][1] for i in range(min(num_summary_sentences, len(ranked_sentences)))]
    summary = ' '.join([s for s in sentences if s in selected_sentences])

    # Adding introduction based on context type
    if context_type == 'story':
        intro = "Once upon a time,"
    elif context_type == 'report':
        intro = "In summary,"
    elif context_type == 'article':
        intro = "To sum it up,"
    else:
        intro = ""

    # Combine introduction and main summary
    final_summary = f"{intro} {summary}"
    return final_summary

# Test the function
test_text = """Once upon a time, in a small town, lived a man named Henry. He was an ordinary man with a monotonous life. One day, he received a mysterious letter. The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. Intrigued but also scared, Henry decided to take the risk and embark on the journey. His journey was filled with peril and uncertainty, but also wonder and excitement. In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. He resumed his ordinary life, but now he was content and grateful for the simple joys that life offered. And so, Henry lived happily ever after."""
important_keywords = ['Henry', 'journey', 'treasure', 'adventure', 'life']  # You would normally use your keyword extraction function here

# Generate the summary
premium_summary = dynamic_detailed_summarize_premium(test_text, detail_level=1, context_type='story', important_keywords=important_keywords)
print("Premium Summary with Detail Level 1:\n")
print(premium_summary)

premium_summary = dynamic_detailed_summarize_premium(test_text, detail_level=2, context_type='story', important_keywords=important_keywords)
print("\n\nPremium Summary with Detail Level 2:\n")
print(premium_summary)


Premium Summary with Detail Level 1:

Once upon a time, The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination Intrigued but also scared, Henry decided to take the risk and embark on the journey Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life


Premium Summary with Detail Level 2:

Once upon a time, Once upon a time, in a small town, lived a man named Henry The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination Intrigued but also scared, Henry decided to take the risk and embark on the journey In the end, he discovered a hidden treasure but realized the real treasure was the journey itself Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.feature_extraction import text as sk_text
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_important_keywords(text, num_keywords=5):
    vectorizer = TfidfVectorizer(stop_words=list(sk_text.ENGLISH_STOP_WORDS))
    X = vectorizer.fit_transform([text])
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:num_keywords]
    print(top_n.tolist())
    return top_n.tolist()

def dynamic_detailed_summarize_premium(input_text, detail_level=1, context_type='general', important_keywords=[]):
    # Preprocessing the text
    sentences = input_text.split('.')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]

    # Add custom stop words or use default English stop words
    my_stop_words = sk_text.ENGLISH_STOP_WORDS

    # Vectorize sentences
    vectorizer = CountVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()

    # Calculate cosine similarity
    cosine_matrix = cosine_similarity(vectors)
    
    # Create a network graph and rank nodes (sentences) by PageRank algorithm
    nx_graph = nx.from_numpy_array(cosine_matrix)
    scores = nx.pagerank(nx_graph)

    # Sort sentences by score
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Select important keywords if not provided
    if not important_keywords:
        important_keywords = extract_important_keywords(input_text)

    # Apply keyword emphasis, increase score of sentences with important keywords
    for i in range(len(ranked_sentences)):
        for keyword in important_keywords:
            if keyword.lower() in ranked_sentences[i][1].lower():
                ranked_sentences[i] = (ranked_sentences[i][0] * 1.2, ranked_sentences[i][1])

    # Re-sort the sentences after keyword emphasis
    ranked_sentences = sorted(ranked_sentences, key=lambda x: x[0], reverse=True)

    # Determine number of sentences based on detail level
    num_sentences = len(sentences) // (3 - detail_level)
    print("NUM SENTENCE IS-:", num_sentences)  # Debugging print statement

    # Generate summary
    summary = [ranked_sentences[i][1] for i in range(num_sentences)]
    
    # Order sentences by their original order in the text
    summary = sorted(summary, key=lambda x: sentences.index(x))
    
    # Create final summary string
    final_summary = ' '.join(summary)
    
    return final_summary

# Test the function
test_text = """Once upon a time, in a small town, lived a man named Henry. He was an ordinary man with a monotonous life. One day, he received a mysterious letter. The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. Intrigued but also scared, Henry decided to take the risk and embark on the journey. His journey was filled with peril and uncertainty, but also wonder and excitement. In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. And he lived happily ever after."""

# Generate two different types of summaries
print("Premium Summary with Detail Level 1:\n")
print(dynamic_detailed_summarize_premium(test_text, detail_level=1, context_type='story'))
print("\n")
print("Premium Summary with Detail Level 2:\n")
print(dynamic_detailed_summarize_premium(test_text, detail_level=2, context_type='story'))


Premium Summary with Detail Level 1:

['journey', 'henry', 'filled', 'small', 'life']
NUM SENTENCE IS-: 4
Intrigued but also scared, Henry decided to take the risk and embark on the journey His journey was filled with peril and uncertainty, but also wonder and excitement In the end, he discovered a hidden treasure but realized the real treasure was the journey itself Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life


Premium Summary with Detail Level 2:

['journey', 'henry', 'filled', 'small', 'life']
NUM SENTENCE IS-: 9
Once upon a time, in a small town, lived a man named Henry He was an ordinary man with a monotonous life One day, he received a mysterious letter The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination Intrigued but also scared, Henry decided to take the risk and embark on the journey His journey was filled with peril and uncertainty, but also wonder and exc

In [9]:
# Let's modify the function to limit the number of sentences in the detailed summary.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
import string
import numpy as np

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

def dynamic_detailed_summarize_premium(text, detail_level=1, context_type='general', important_keywords=[]):
    # Tokenize the sentences
    sentences = sent_tokenize(text)
    
    # Set maximum number of sentences for the detailed summary
    max_sentences = len(sentences) if detail_level == 2 else int(len(sentences) / 2)
    
    # Tokenize the words and create a dictionary of word frequencies
    words = word_tokenize(text.lower())
    freq = {}
    for word in words:
        freq[word] = freq.get(word, 0) + 1
    
    # Score each sentence based on the sum of the frequencies of its words
    scores = {}
    for sentence in sentences:
        scores[sentence] = 0
        for word in word_tokenize(sentence.lower()):
            if word in freq:
                scores[sentence] += freq[word]
    
    # Add bonus for presence of important keywords
    for keyword in important_keywords:
        for sentence in sentences:
            if keyword.lower() in sentence.lower():
                scores[sentence] += 5
    
    # Sort the sentences by their scores and select top sentences
    sorted_sentences = [sentence for sentence, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)]
    
    # Limit the number of sentences based on the detail level
    sorted_sentences = sorted_sentences[:max_sentences]
    
    # Re-order the selected sentences to maintain their original order in the text
    final_summary = [sentence for sentence in sentences if sentence in sorted_sentences]
    
    return ' '.join(final_summary)

# Test the function
test_text = """Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. 
One day, he received a mysterious letter. 
The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. 
Intrigued but also scared, Henry decided to take the risk and embark on the journey. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. 
And he lived happily ever after."""

print("Premium Summary with Detail Level 1:\n")
print(dynamic_detailed_summarize_premium(test_text, detail_level=1, important_keywords=['journey', 'treasure']))

print("\nPremium Summary with Detail Level 2:\n")
print(dynamic_detailed_summarize_premium(test_text, detail_level=2, important_keywords=['journey', 'treasure']))


Premium Summary with Detail Level 1:

Once upon a time, in a small town, lived a man named Henry. Intrigued but also scared, Henry decided to take the risk and embark on the journey. In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life.

Premium Summary with Detail Level 2:

Once upon a time, in a small town, lived a man named Henry. He was an ordinary man with a monotonous life. One day, he received a mysterious letter. The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. Intrigued but also scared, Henry decided to take the risk and embark on the journey. His journey was filled with peril and uncertainty, but also wonder and excitement. In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. Filled with newfound wisdom and experience,

In [10]:
def extract_important_keywords(text, num_keywords=5):
    vectorizer = TfidfVectorizer(stop_words=list(sk_text.ENGLISH_STOP_WORDS))
    X = vectorizer.fit_transform([text])
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:num_keywords]
    print(top_n.tolist())
    return top_n.tolist()

In [11]:
extract_important_keywords(test_text,10)

['journey', 'henry', 'filled', 'small', 'life', 'letter', 'lived', 'man', 'treasure', 'town']


['journey',
 'henry',
 'filled',
 'small',
 'life',
 'letter',
 'lived',
 'man',
 'treasure',
 'town']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np

def extract_important_keywords(text, num_keywords=10):
    # Tokenization and Vectorization
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
    
    # Fit and transform the text to generate TF-IDF features
    X = vectorizer.fit_transform([text])
    
    # Dimensionality reduction using TruncatedSVD (aka LSA: Latent Semantic Analysis)
    svd = TruncatedSVD(n_components=1, random_state=42)
    svd.fit(X)
    
    # Sort the features by importance
    terms = vectorizer.get_feature_names_out()
    sorted_indices = np.argsort(svd.components_).flatten()[::-1]
    
    # Extract top features
    top_keywords = [terms[i] for i in sorted_indices[:num_keywords]]
    
    return top_keywords

# Test the function with the sample text
test_text = """Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. 
One day, he received a mysterious letter. 
The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. 
Intrigued but also scared, Henry decided to take the risk and embark on the journey. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. 
And he lived happily ever after."""

important_keywords_refined = extract_important_keywords(test_text)
important_keywords_refined


c:\Users\RohanRVC\Documents\Business\E-AI-Service\.AI_service\Lib\site-packages\sklearn\decomposition\_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


['journey',
 'henry',
 'man',
 'treasure',
 'lived',
 'life',
 'letter',
 'journey filled',
 'small',
 'small town']

In [13]:
# Implementing the premium feature with refined keyword extraction method
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text 
import networkx as nx
import re

# Stop words list
my_stop_words = list(text.ENGLISH_STOP_WORDS)

# Function to extract important keywords from text
def extract_important_keywords(text, num_keywords=10):
    vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=my_stop_words).fit([text])
    transformed_text = vectorizer.transform([text])
    keywords = sorted([(word, transformed_text[0, idx]) for word, idx in vectorizer.vocabulary_.items()], key=lambda x: x[1], reverse=True)[:num_keywords]
    refined_keywords = [k[0] for k in keywords]
    return refined_keywords

# Function to generate a premium summary
def premium_summary(text, detail_level=1, context_type='generic', important_keywords=None):
    # Extract sentences
    sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', text)
    
    # Extract important keywords if not provided
    if not important_keywords:
        important_keywords = extract_important_keywords(text)
    
    print(f"Important Keywords: {important_keywords}")

    # Vectorization
    vectorizer = CountVectorizer(ngram_range=(1, 2), vocabulary=important_keywords).fit_transform(sentences)

    # Cosine Similarity Matrix
    cosine_matrix = cosine_similarity(vectorizer)

    # Ranking sentences
    nx_graph = nx.from_numpy_array(cosine_matrix)
    scores = nx.pagerank(nx_graph)

    # Sort the sentence rankings
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    # Determine the number of sentences based on detail level
    num_sentences = len(sentences) // (5 - detail_level)
    print(f"NUM SENTENCE IS-: {num_sentences}")
    
    # Extract the summary
    summary = [ranked_sentences[i][1] for i in range(num_sentences)]
    
    # Re-order summary based on their appearance in the original text
    ordered_summary = sorted(summary, key=lambda x: sentences.index(x))
    
    return ' '.join(ordered_summary)

# Test Text
test_text = """Once upon a time, in a small town, lived a man named Henry. He was an ordinary man with a monotonous life. One day, he received a mysterious letter. The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. Intrigued but also scared, Henry decided to take the risk and embark on the journey. His journey was filled with peril and uncertainty, but also wonder and excitement. He faced numerous challenges and overcame many obstacles. In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. And he lived happily ever after."""

# Generate summaries with different detail levels
summary_level_1 = premium_summary(test_text, detail_level=1, context_type='story')
summary_level_2 = premium_summary(test_text, detail_level=2, context_type='story')

(summary_level_1, summary_level_2)


Important Keywords: ['henry', 'journey', 'small', 'town', 'lived', 'man', 'life', 'letter', 'filled', 'treasure']
NUM SENTENCE IS-: 2
Important Keywords: ['henry', 'journey', 'small', 'town', 'lived', 'man', 'life', 'letter', 'filled', 'treasure']
NUM SENTENCE IS-: 3


('Once upon a time, in a small town, lived a man named Henry. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life.',
 'Once upon a time, in a small town, lived a man named Henry. Intrigued but also scared, Henry decided to take the risk and embark on the journey. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life.')

In [14]:
# Let's use a longer example text to test the function
long_example_text = """
In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. He was a man of wisdom and knowledge, 
but he was also a recluse who rarely ventured out of his store. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science. It was a treasure trove for book lovers, but it was always empty, as not many people knew about it.

One fine day, a young girl named Emily walked into the store. She was an avid reader and was looking for something new to read. 
Mr. Johnson welcomed her warmly and suggested several books. Emily was captivated by the variety and ended up spending hours in the store. 
She finally chose a book on ancient civilizations and went to the counter to pay for it.

As Mr. Johnson was wrapping the book, he told Emily about the special book hidden in the store. 
It was said to contain the secrets of the universe and was written by a mysterious author. Intrigued, Emily asked if she could see the book. 
Mr. Johnson hesitated for a moment but then led her to a hidden room at the back of the store. 
He took out a key from his pocket and opened a small, rusty safe. Inside was the special book, covered in dust and cobwebs.

Emily was fascinated and asked if she could buy the book. Mr. Johnson said that the book was not for sale, 
but he would let her read it in the store. Emily agreed and started reading the book. 
As she turned the pages, she felt a strange sense of enlightenment. 
The book was filled with profound wisdom and insights that she had never encountered before.

After reading the book, Emily felt like a completely different person. 
She thanked Mr. Johnson and promised to come back for more books. As she left the store, she felt an overwhelming sense of gratitude. 
She had discovered a hidden gem in the heart of the city, and it had changed her life forever.

Days turned into weeks, and weeks into months. Emily became a regular at the bookstore, 
and Mr. Johnson became her mentor. They spent hours discussing various subjects, and Emily felt her knowledge and understanding deepen. 
One day, Mr. Johnson fell ill and had to close the store for a few days. Emily was concerned and visited him at his home. 
She found out that Mr. Johnson was suffering from a rare illness and did not have much time left.

Heartbroken, Emily decided to take over the bookstore and continue Mr. Johnson's legacy. 
She managed the store with the same passion and dedication that he had shown. The bookstore flourished under her management, 
and it became a popular spot for book lovers. Emily introduced various new sections and even started selling online. 
The special book was placed in a glass case for everyone to see, but it was still not for sale.

Years later, Emily was awarded the 'Entrepreneur of the Year' for her incredible work. 
She dedicated the award to Mr. Johnson, who had passed away but had left a lasting impact on her life. 
The bookstore continued to thrive, and Emily never forgot the lessons she had learned from the special book and Mr. Johnson.
"""

# Extracting important keywords from the long example text
important_keywords_long_text = extract_important_keywords(long_example_text, num_keywords=5)
print(f"Important Keywords: {important_keywords_long_text}")

# Generating summaries with different detail levels
summary_detail_1 = dynamic_detailed_summarize_premium(long_example_text, detail_level=1, context_type='story', important_keywords=important_keywords_long_text)
summary_detail_2 = dynamic_detailed_summarize_premium(long_example_text, detail_level=2, context_type='story', important_keywords=important_keywords_long_text)

print("\nPremium Summary with Detail Level 1:\n")
print(summary_detail_1)

print("\nPremium Summary with Detail Level 2:\n")
print(summary_detail_2)


Important Keywords: ['book', 'emily', 'mr', 'johnson', 'mr johnson']

Premium Summary with Detail Level 1:


In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. He was a man of wisdom and knowledge, 
but he was also a recluse who rarely ventured out of his store. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science. It was a treasure trove for book lovers, but it was always empty, as not many people knew about it. Emily was captivated by the variety and ended up spending hours in the store. She finally chose a book on ancient civilizations and went to the counter to pay for it. As Mr. Johnson was wrapping the book, he told Emily about the special book hidden in the store. It was said to contain the secrets of the universe and was written by a mysterious author. Mr. Johnson hesitated for a moment but then led her to a hidden room at the back of the store. Mr. Johnson said that the book was no

In [15]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Correcting the min_df parameter and re-running the function

def extract_important_keywords_v3(input_text, num_keywords=5):
    # List of stopwords
    my_stop_words = ENGLISH_STOP_WORDS.union(['s', 'mr'])
    
    # Initialize the TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words=list(my_stop_words), min_df=1)
    
    # Fit and transform the text
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    
    # Get the feature names
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    
    # Get the tf-idf values for the first document (our text)
    tfidf_values = tfidf_matrix.toarray()[0]
    
    # Get the indices sorted by tf-idf values
    sorted_indices = tfidf_values.argsort()[::-1]
    
    # Get the top num_keywords
    top_keywords = feature_names[sorted_indices][:num_keywords]
    
    return top_keywords.tolist()

henry="""Once upon a time, in a small town, lived a man named Henry. 
He was an ordinary man with a monotonous life. One day, he received a mysterious letter. 
The letter was an invitation to a grand adventure, promising treasures and wonders beyond imagination. 
Intrigued but also scared, Henry decided to take the risk and embark on the journey. 
His journey was filled with peril and uncertainty, but also wonder and excitement. 
In the end, he discovered a hidden treasure but realized the real treasure was the journey itself. 
Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. 
And he lived happily ever after."""
test_text3="""In a small, picturesque town nestled in the mountains, there lived a young woman named Lily. She was known throughout the village for her exceptional talent for painting. Her canvases were filled with vibrant colors and intricate details, capturing the beauty of the world around her.

But what made Lily's paintings truly special was their ability to come to life. When she painted a lush forest, the leaves rustled, and the birds sang. When she painted a tranquil lake, the water rippled, and the fish swam. Her art had a magic all its own.

One sunny afternoon, a group of children gathered outside Lily's cottage, eager to see her latest masterpiece. With a smile, she unveiled a canvas depicting a magical garden filled with fantastical creatures and enchanted plants. As the children gazed in awe, the garden began to stir to life. Gnomes danced, unicorns pranced, and fireflies illuminated the scene.

Lily's gift became renowned far and wide. Visitors from distant lands traveled to the village to witness the enchanting paintings. However, her fame also brought forth a challenge. A wealthy collector named Victor, who lived in the nearby city, offered Lily a fortune for her latest creation. But Victor had a reputation for hoarding art, keeping it hidden from the world.

Lily refused Victor's offer. She believed that her art should be shared, not hidden away. This decision angered Victor, and he resorted to theft. In the dead of night, he stole Lily's magical garden painting and locked it away in his vast, private gallery.

Heartbroken but determined, Lily hatched a plan to rescue her masterpiece. She painted a new scene - a forest filled with wise owls and cunning foxes. This time, she included a message within the painting, a plea for help to the creatures of the night.

As the moon rose, the owls and foxes saw Lily's message and, understanding her distress, rallied to her aid. They embarked on a daring mission to retrieve the stolen painting. With their clever strategies, they outwitted Victor's guards and returned the artwork to its rightful place.

The magical garden painting was once again displayed in Lily's cottage, and the enchantment it brought to the village was more wondrous than ever. But Victor learned a valuable lesson about the true essence of art - it was meant to be shared and appreciated by all, not locked away in isolation.

The village celebrated Lily's courage and creativity, and she continued to paint masterpieces that brought joy and enchantment to all who saw them. Her art not only came to life on canvas but also in the hearts of those who were touched by its magic."""
test_text2 = """
In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. He was a man of wisdom and knowledge, 
but he was also a recluse who rarely ventured out of his store. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science. It was a treasure trove for book lovers, but it was always empty, as not many people knew about it. 
One fine day, a young girl named Emily walked into the store. She was an avid reader and was looking for something new to read. 
Mr. Johnson welcomed her warmly and suggested several books. Emily was captivated by the variety and ended up spending hours in the store. 
She finally chose a book on ancient civilizations and went to the counter to pay for it. As Mr. Johnson was wrapping the book, he told Emily about the special book hidden in the store. 
It was said to contain the secrets of the universe and was written by a mysterious author. Intrigued, Emily asked if she could see the book. 
Mr. Johnson hesitated for a moment but then led her to a hidden room at the back of the store. He took out a key from his pocket and opened a small, rusty safe. 
Inside was the special book, covered in dust and cobwebs. Emily was fascinated and asked if she could buy the book. Mr. Johnson said that the book was not for sale, 
but he would let her read it in the store. Emily agreed and started reading the book. As she turned the pages, she felt a strange sense of enlightenment. 
The book was filled with profound wisdom and insights that she had never encountered before. After reading the book, Emily felt like a completely different person. 
She thanked Mr. Johnson and promised to come back for more books. As she left the store, she felt an overwhelming sense of gratitude. 
She had discovered a hidden gem in the heart of the city, and it had changed her life forever. Days turned into weeks, and weeks into months. 
Emily became a regular at the bookstore, and Mr. Johnson became her mentor. They spent hours discussing various subjects, and Emily felt her knowledge and understanding deepen. 
One day, Mr. Johnson fell ill and had to close the store for a few days. Emily was concerned and visited him at his home. 
She found out that Mr. Johnson was suffering from a rare illness and did not have much time left. Heartbroken, Emily decided to take over the bookstore and continue Mr. Johnson's legacy. 
She managed the store with the same passion and dedication that he had shown. The bookstore flourished under her management, and it became a popular spot for book lovers. 
Emily introduced various new sections and even started selling online. The special book was placed in a glass case for everyone to see, but it was still not for sale. 
Years later, Emily was awarded the 'Entrepreneur of the Year' for her incredible work. She dedicated the award to Mr. Johnson, who had passed away but had left a lasting impact on her life. 
The bookstore continued to thrive, and Emily never forgot the lessons she had learned from the special book and Mr. Johnson.
"""
tt="""In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. 
He was a man of wisdom and knowledge, but he was also a recluse who rarely ventured out of his store. 
The bookstore was filled with old, dusty books on every subject imaginable, from ancient history to modern science. 
It was a treasure trove for book lovers, but it was always empty, as not many people knew about it."""
# Extract important keywords from the text
refined_keywords = extract_important_keywords_v3(henry)
refined_keywords


['journey', 'henry', 'filled', 'small', 'life']

In [16]:
def dynamic_detailed_summarize_premium(text, detail_level=1, important_keywords=None):
    # Split the text into sentences
    sentences = text.split('. ')
    num_sentences = len(sentences)
    
    # Extract important keywords if not provided
    if not important_keywords:
        important_keywords = extract_important_keywords_v3(text)
    
    print(f"Important Keywords: {important_keywords}")
    
    # Score each sentence based on the presence of important keywords
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_scores[i] = 0
        for keyword in important_keywords:
            if keyword.lower() in sentence.lower():
                sentence_scores[i] += 1
    
    # Sort sentences by score
    sorted_sentences = [sentences[i] for i, _ in sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)]
    
    # Decide the number of sentences to include based on the detail level
    num_include = int(np.ceil(num_sentences * (0.2 * detail_level)))
    if num_include > num_sentences:
        num_include = num_sentences
    
    print(f"NUM SENTENCE IS-: {num_include}")
    
    # Create the summary
    summary = '. '.join(sorted_sentences[:num_include])
    
    return summary


In [17]:
# Generate summaries with varying detail levels
summary1 = dynamic_detailed_summarize_premium(test_text, detail_level=1)
summary2 = dynamic_detailed_summarize_premium(test_text, detail_level=2)

print(f"\nPremium Summary with Detail Level 1:\n\n{summary1}\n")
print(f"\nPremium Summary with Detail Level 2:\n\n{summary2}\n")

Important Keywords: ['journey', 'henry', 'lived', 'life', 'letter']
NUM SENTENCE IS-: 2
Important Keywords: ['journey', 'henry', 'lived', 'life', 'letter']
NUM SENTENCE IS-: 4

Premium Summary with Detail Level 1:

Once upon a time, in a small town, lived a man named Henry. Intrigued but also scared, Henry decided to take the risk and embark on the journey


Premium Summary with Detail Level 2:

Once upon a time, in a small town, lived a man named Henry. Intrigued but also scared, Henry decided to take the risk and embark on the journey. Filled with newfound wisdom and experience, Henry returned to his small town, richer not just in wealth but in life. He was an ordinary man with a monotonous life



In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from nltk.tokenize import sent_tokenize

# Function to extract important keywords
def extract_important_keywords(input_text, num_keywords=5):
    my_stop_words = ENGLISH_STOP_WORDS.union(['s', 'mr'])
    tfidf_vectorizer = TfidfVectorizer(stop_words=list(my_stop_words), min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    tfidf_values = tfidf_matrix.toarray()[0]
    sorted_indices = tfidf_values.argsort()[::-1]
    top_keywords = feature_names[sorted_indices][:num_keywords]
    print(f'Top {num_keywords} keywords are-:{top_keywords.tolist()}')
    return top_keywords.tolist()

# Function to generate dynamic detailed summaries
def dynamic_detailed_summarize_premium(input_text, detail_level=1, context_type='general', important_keywords=None):
    if important_keywords is None:
        important_keywords = extract_important_keywords(input_text)
    
    sentences = sent_tokenize(input_text)
    num_sentences = len(sentences)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()
    related_sentences_indices = cosine_similarities.argsort()[:-num_sentences:-1]
    
    important_sentences = [sentences[i] for i in related_sentences_indices if any(kw in sentences[i].lower() for kw in important_keywords)]
    
    # Context weightage for adjusting the number of sentences based on context_type
    context_weightage = {
        'general': 1,
        'story': 1.5,
        'report': 1.2,
        'article': 1.3  # Adding support for 'article' context
    }
    
    num_output_sentences = max(2, int(detail_level * len(important_sentences) / 5 * context_weightage.get(context_type, 1)))
    selected_sentences = important_sentences[:num_output_sentences]
    
    return ' '.join(selected_sentences)

# Test example
test_text = """
In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. He was a man of wisdom and knowledge, 
but he was also a recluse who rarely ventured out of his store. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science. It was a treasure trove for book lovers, but it was always empty, as not many people knew about it.
"""

# Extract keywords
important_keywords = extract_important_keywords(test_text)
print("Important Keywords:", important_keywords)

# Generate Premium Summary with Detail Level 1
summary1 = dynamic_detailed_summarize_premium(test_text, detail_level=1, context_type='article', important_keywords=important_keywords)
print("\nPremium Summary with Detail Level 1:\n")
print(summary1)

# Generate Premium Summary with Detail Level 2
summary2 = dynamic_detailed_summarize_premium(test_text, detail_level=2, context_type='article', important_keywords=important_keywords)
print("\nPremium Summary with Detail Level 2:\n")
print(summary2)


Top 5 keywords are-:['bookstore', 'wisdom', 'ventured', 'book', 'books']
Important Keywords: ['bookstore', 'wisdom', 'ventured', 'book', 'books']

Premium Summary with Detail Level 1:


In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science.

Premium Summary with Detail Level 2:


In the heart of the city, there was a small, obscure bookstore owned by Mr. Johnson. The bookstore was filled with old, dusty books on every subject imaginable, 
from ancient history to modern science.


In [19]:
# Test story for the refined keyword extraction and summarization function
test_story_2 = """
Sarah was a brilliant scientist working in the field of renewable energy. She was dedicated to finding solutions to the 
climate crisis and had spent years researching solar and wind energy. Sarah's lab was filled with prototypes, designs, and 
models of new energy systems. However, her work was not getting the attention it deserved. Government funding was limited, 
and most investors were skeptical about renewable energy. Sarah was on the verge of giving up when she received an email 
from a renowned environmental organization. They were interested in her research and wanted to offer her a grant. Filled 
with renewed hope, Sarah accepted the offer and continued her work with more vigor than ever. Over the next few months, 
Sarah's research gained traction, and her innovations started receiving media coverage. Her breakthrough came when she 
developed a compact solar panel that could generate twice as much energy as traditional panels. This innovation caught 
the attention of major companies, and soon, Sarah's technology was being implemented on a large scale. Her work played 
a crucial role in reducing carbon emissions and combatting climate change. Sarah became a leading figure in the field 
of renewable energy and was awarded numerous accolades for her contributions.
"""

# Extracting important keywords from the story
important_keywords_v3 = extract_important_keywords_v3(test_story_2, num_keywords=5)

# Creating Premium Summaries for the test story
premium_summary_1_v3 = dynamic_detailed_summarize_premium(test_story_2, detail_level=1, context_type='article', important_keywords=important_keywords_v3)
premium_summary_2_v3 = dynamic_detailed_summarize_premium(test_story_2, detail_level=2, context_type='article', important_keywords=important_keywords_v3)

important_keywords_v3, premium_summary_1_v3, premium_summary_2_v3


(['sarah', 'energy', 'renewable', 'work', 'filled'],
 '\nSarah was a brilliant scientist working in the field of renewable energy. Sarah became a leading figure in the field \nof renewable energy and was awarded numerous accolades for her contributions.',
 "\nSarah was a brilliant scientist working in the field of renewable energy. Sarah became a leading figure in the field \nof renewable energy and was awarded numerous accolades for her contributions. Government funding was limited, \nand most investors were skeptical about renewable energy. Sarah's lab was filled with prototypes, designs, and \nmodels of new energy systems. She was dedicated to finding solutions to the \nclimate crisis and had spent years researching solar and wind energy.")

In [20]:
# Additional examples to test the dynamic detailed summarize premium function
example_1 = """
John was a dedicated physician who had always been fascinated by the complexities of the human brain. 
After years of research, he discovered a groundbreaking treatment for Alzheimer's disease. 
Despite initial skepticism from the medical community, he conducted multiple clinical trials which proved to be successful. 
His work was published in reputed medical journals and he received numerous awards for his contribution to medicine. 
John continued to work tirelessly, aiming to make the treatment accessible to people around the world.
"""

example_2 = """
Emily was an aspiring musician who had a deep passion for classical music. 
She practiced for hours every day, aiming to perfect her technique. 
Emily faced numerous challenges, including financial constraints and lack of proper training facilities. 
However, she never gave up and continued to pursue her dreams. 
Her perseverance paid off when she won a national music competition, earning her a scholarship to a prestigious music academy. 
Emily's journey was an inspiration to many, proving that dedication and hard work can overcome any obstacle.
"""

example_3 = """
In a small town lived a humble baker named Tim. He was well-known for his delicious pastries and cakes. 
One day, a food critic visited his bakery and wrote a stellar review. 
Soon, people from neighboring towns began visiting his bakery. 
Though Tim was thrilled with the newfound fame, he also felt overwhelmed with the increase in demand. 
He decided to expand his business and hired additional staff. 
Within a year, Tim's bakery became the go-to place for all things sweet, and he lived happily, cherishing his hard-earned success.
"""

# Run the function on these examples
example_1_keywords = dynamic_detailed_summarize_premium(example_1, detail_level=1, context_type='report')
example_2_keywords  = dynamic_detailed_summarize_premium(example_2, detail_level=1, context_type='story')
example_3_keywords= dynamic_detailed_summarize_premium(example_3, detail_level=1, context_type='story')

example_1_keywords,  example_2_keywords, example_3_keywords


Top 5 keywords are-:['john', 'work', 'treatment', 'medical', 'continued']
Top 5 keywords are-:['music', 'emily', 'work', 'including', 'hard']
Top 5 keywords are-:['bakery', 'tim', 'lived', 'demand', 'humble']


('\nJohn was a dedicated physician who had always been fascinated by the complexities of the human brain. John continued to work tirelessly, aiming to make the treatment accessible to people around the world.',
 '\nEmily was an aspiring musician who had a deep passion for classical music. Her perseverance paid off when she won a national music competition, earning her a scholarship to a prestigious music academy.',
 "\nIn a small town lived a humble baker named Tim. Within a year, Tim's bakery became the go-to place for all things sweet, and he lived happily, cherishing his hard-earned success.")

In [21]:
dynamic_detailed_summarize_premium(test_text3,detail_level=1,context_type='story')

Top 5 keywords are-:['lily', 'victor', 'art', 'painting', 'garden']


"In a small, picturesque town nestled in the mountains, there lived a young woman named Lily. A wealthy collector named Victor, who lived in the nearby city, offered Lily a fortune for her latest creation. Lily refused Victor's offer. Lily's gift became renowned far and wide. But what made Lily's paintings truly special was their ability to come to life. Heartbroken but determined, Lily hatched a plan to rescue her masterpiece."

In [22]:
reportss="""According to the latest research by the National Health Institute, exercise plays a crucial role in mental well-being. 
The study surveyed 10,000 adults between the ages of 25 and 60. The respondents were categorized into three groups based 
on their exercise habits: low, moderate, and high. The research found a significant correlation between the amount of 
exercise and levels of stress, anxiety, and depression. Those in the 'high' exercise group reported 30% lower levels of 
stress compared to the 'low' exercise group. The study strongly recommends incorporating at least 30 minutes of moderate 
exercise into daily routines for better mental health.
"""
dynamic_detailed_summarize_premium(reportss,detail_level=2,context_type='report')

Top 5 keywords are-:['exercise', 'high', 'research', 'health', 'study']


'According to the latest research by the National Health Institute, exercise plays a crucial role in mental well-being. The study strongly recommends incorporating at least 30 minutes of moderate \nexercise into daily routines for better mental health.'

In [23]:
General="Technology has been advancing at an unprecedented rate, transforming the way we live, work, and interact. From smartphones to self-driving cars, technological innovations have made our lives more convenient. However, there are concerns about privacy, data security, and the ethical implications of certain technologies. Despite these challenges, technology continues to evolve, presenting new opportunities and risks."

Story="Once upon a time, in a small village, lived a young girl named Lily. She was known for her kindness and bravery. The village was plagued by a fierce dragon that would come every month to take livestock and scare the villagers. Tired of living in fear, Lily decided to confront the dragon. Armed with her intelligence and a magical sword, she went into the mountain where the dragon lived. After a long and perilous journey, she found the dragon and discovered that it was only stealing because it was trying to feed its young ones. Using her intelligence, Lily made a pact with the dragon, ending years of terror for her village. She returned a hero, and they all lived happily ever after."

Report="According to a recent report by the World Health Organization, the rate of obesity has tripled worldwide in the last 40 years. The report suggests that lack of physical activity and unhealthy eating habits are the main contributors to this epidemic. Governments are advised to implement policies promoting physical exercise and better eating habits. The report emphasizes that immediate actions are required to curb this growing health crisis."

Article="In the last decade, renewable energy has emerged as a viable alternative to fossil fuels. With the looming climate crisis, it's more important than ever to shift towards sustainable energy sources. Solar and wind energy, in particular, have seen significant advancements. Government policies are also increasingly supportive of renewable energy initiatives. However, challenges such as storage and distribution remain. This article explores the potential of renewable energy and the obstacles that need to be overcome."
g=dynamic_detailed_summarize_premium(General,detail_level=2,context_type='general')
s=dynamic_detailed_summarize_premium(Story,detail_level=2,context_type='story')
a=dynamic_detailed_summarize_premium(Article,detail_level=2,context_type='article')
r=dynamic_detailed_summarize_premium(Report,detail_level=2,context_type='report')
g,s,a,r

Top 5 keywords are-:['technology', 'work', 'data', 'innovations', 'implications']
Top 5 keywords are-:['dragon', 'lived', 'lily', 'village', 'young']
Top 5 keywords are-:['energy', 'renewable', 'wind', 'initiatives', 'important']
Top 5 keywords are-:['report', 'habits', 'eating', 'physical', 'health']


('Technology has been advancing at an unprecedented rate, transforming the way we live, work, and interact. Despite these challenges, technology continues to evolve, presenting new opportunities and risks.',
 'Once upon a time, in a small village, lived a young girl named Lily. Using her intelligence, Lily made a pact with the dragon, ending years of terror for her village. She returned a hero, and they all lived happily ever after. After a long and perilous journey, she found the dragon and discovered that it was only stealing because it was trying to feed its young ones.',
 'In the last decade, renewable energy has emerged as a viable alternative to fossil fuels. Government policies are also increasingly supportive of renewable energy initiatives.',
 'According to a recent report by the World Health Organization, the rate of obesity has tripled worldwide in the last 40 years. The report emphasizes that immediate actions are required to curb this growing health crisis.')

In [24]:
article2="""Anysphere, a startup building what it describes as an “AI-native” software development environment, called Cursor, today announced that it raised $8 million in seed funding led by OpenAI’s Startup Fund with participation from former GitHub CEO Nat Friedman, Dropbox co-founder Arash Ferdowsi and other angel investors.

The new cash, which brings Anysphere’s total raised to $11 million, will be put toward hiring and supporting Anysphere’s AI and machine learning research, co-founder and CEO Michael Truell said

“In the next several years, our mission is to make programming an order of magnitude faster, more fun and creative,” Truell told TechCrunch in an email interview. “Our platform enables all developers to build software faster.”

Truell met Anysphere’s other co-founders, Sualeh Asif, Arvid Lunnemark and Aman Sanger, while at MIT, where they became close friends. The four shared the goal of creating an integrated development environment (IDE) that could speed up common programming and software building tasks, like debugging.

To that end, Cursor, which is a fork of VS Code, Microsoft’s open source code editor, packs AI-powered tools designed to help developers write — and ask questions about — code. Cursor can respond to queries like “What service in VS Code lets me save a state to disk,” for instance, and pull up relevant documentation and code definitions as programmers work.

Cursor also features generative AI capabilities powered by OpenAI models, namely the ability to generate code from a prompt. And it can passively scan files and surface potential bugs in codebases.

“When people think ‘AI plus coding,’ their mind usually goes to AI-powered autocomplete,” Sanger said via email. “We think this has been done particularly well by GitHub Copilot and others, so we’re focused on features that come after autocomplete, like finding and fixing bugs and codebase Q&A.”

Does Cursor have much hope of competing with the incumbents in the IDE space, though? That’s a reasonable question. According to StackOverflow’s 2023 Developer Survey, Microsoft’s Visual Studio Code remains far and away the most popular IDE, with ~73% of developers saying that it’s their go-to. The Anysphere team, indeed, sees Microsoft is their main competitor. And they acknowledge that the tech giant has a substantial distribution advantage. But they make the case that, because Visual Studio Code has a wide and varied customer base, Microsoft can’t make radical changes or ship major upgrades very quickly without risking alienating a portion of its users.

“The ceiling in the AI coding space is so high — there’s so much to do — that it’s not possible to just clone the tech and then put great sales on top,” Truell said. “You need to constantly evolve the tech. There’s over 26 million developers around the world, and there’s a huge market for those that want a truly AI-native experience.”

The five-person Anysphere team is nothing if not ambitious, with a host of features they hope to get to on the development roadmap for Cursor. In the coming months, the plan is to enable Cursor to make more complex edits across files and entire folders, improve at finding code and learn new libraries from documentation.

In the meantime, Anysphere’s popularity is slowly growing, Truell claims, with tens of thousands of people on the platform and a “fast-growing” paying customer base. Annual recurring revenue is already over $1 million — an auspicious start for a roughly-one-year-old company.

“For now, we’re focused on the individual and teams experience over Enterprise,” Sanger said. “In the long term, we believe Cursor will be a no-brainer for enterprises, given the massive boost in developer productivity.” """
dynamic_detailed_summarize_premium(article2,1,'article')

Top 5 keywords are-:['code', 'cursor', 'ai', 'anysphere', 'truell']


'Anysphere, a startup building what it describes as an “AI-native” software development environment, called Cursor, today announced that it raised $8 million in seed funding led by OpenAI’s Startup Fund with participation from former GitHub CEO Nat Friedman, Dropbox co-founder Arash Ferdowsi and other angel investors. There’s over 26 million developers around the world, and there’s a huge market for those that want a truly AI-native experience.”\n\nThe five-person Anysphere team is nothing if not ambitious, with a host of features they hope to get to on the development roadmap for Cursor. The new cash, which brings Anysphere’s total raised to $11 million, will be put toward hiring and supporting Anysphere’s AI and machine learning research, co-founder and CEO Michael Truell said\n\n“In the next several years, our mission is to make programming an order of magnitude faster, more fun and creative,” Truell told TechCrunch in an email interview. Cursor also features generative AI capabilit

In [25]:
article3="""TikTok today is introducing a new feature that will allow its users to post directly to the video platform from a range of popular editing apps, including Adobe’s AI-powered video editing software Premiere Pro, its AI creativity app Adobe Express, as well as others, including Twitch, SocialPilot and ByteDance’s CapCut. The new offering, Direct Post, is actually aimed at the developers of third-party apps that want to integrate more closely with TikTok and builds on TikTok’s existing “Share to” integrations which allowed third-party apps to publish to TikTok along with their own hashtags.

With Direct Post, however, apps not only have the ability to post video content directly to TikTok’s platform, they can also take advantage of other options, like the ability to set captions, audience settings and more within their own platform, then send information through to TikTok with a single click. In addition, the feature allows long-form video creators to schedule their content to publish to TikTok through social media management platforms.

The addition will make it possible for TikTok to benefit from the advances in the wider creativity app landscape, including those apps that are leveraging AI technology as part of the video editing process.

To use the feature, creators authenticate with their TikTok account within the third-party app. The apps that have partnered with TikTok are also vetted through an auditing process before being allowed to use Direct Post’s Content Posting API. 

Adobe, which just today unveiled a host of new AI software and technology at its annual MAX conference, is a key partner for TikTok’s new feature. The company will now be offering Direct Post from Adobe Premiere Pro, which offers AI-powered video editing tools, and Adobe Express, a lightweight AI-powered creativity app.

“Now more than ever, publishing content in real-time has become a necessity, and creators of all skill levels need tools that can empower them with greater efficiency and without constraints,” said Deepa Subramaniam, Adobe VP of Creative Cloud product marketing, in a statement about the launch. “With the new Direct Post feature for TikTok available in Adobe Express and Premiere Pro, creators can continue to create standout content, but with increased speed and without any interruption to their creative workflows.”

Other partners include CapCut, a popular video editing tool from TikTok’s parent company ByteDance, which is also the second app from the company to hit the $100 million milestone, data.ai recently reported.

In addition, Twitch streamers will be able to use Twitch’s Clip Editor to covert their clips to portrait mode for direct sharing on TikTok and will also have the option to export their portrait clip to TikTok as a draft, so they can continue their edits.

The social media marketing tool SocialPilot is adopting Direct Post, as well, bringing the option to professionals, teams, agencies and businesses that need to automate their social media posting.

Access to the API is open to developers of creative tools and video editing apps. The API will soon support photos, too, TikTok notes."""
dynamic_detailed_summarize_premium(article3,1,'article')

Top 5 keywords are-:['tiktok', 'post', 'video', 'apps', 'ai']


'TikTok today is introducing a new feature that will allow its users to post directly to the video platform from a range of popular editing apps, including Adobe’s AI-powered video editing software Premiere Pro, its AI creativity app Adobe Express, as well as others, including Twitch, SocialPilot and ByteDance’s CapCut. The company will now be offering Direct Post from Adobe Premiere Pro, which offers AI-powered video editing tools, and Adobe Express, a lightweight AI-powered creativity app. “With the new Direct Post feature for TikTok available in Adobe Express and Premiere Pro, creators can continue to create standout content, but with increased speed and without any interruption to their creative workflows.”\n\nOther partners include CapCut, a popular video editing tool from TikTok’s parent company ByteDance, which is also the second app from the company to hit the $100 million milestone, data.ai recently reported.'

In [26]:
# Here's a long example of a news article.
news_article = """In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) 
have found water on Mars. The discovery was made possible by the latest Mars rover, named Curiosity 2.0, 
which landed on the red planet last year. The rover was equipped with advanced sensors and drilling equipment, 
allowing it to collect samples from the Martian soil. After months of analysis, the ISRI team confirmed that 
they have found traces of liquid water.

The discovery has significant implications for future space missions, including the possibility of human 
colonization of Mars. "This is a monumental step forward in our quest to find life beyond Earth," said Dr. Sarah Johnson, 
the lead scientist on the project. She further added, "Water is essential for life as we know it, and its presence 
on Mars opens up new avenues for exploration."

However, the discovery also poses several challenges, including the ethical implications of colonizing another planet. 
Some experts argue that we should focus on addressing the environmental issues on Earth before venturing into space. 
"While this is an incredible scientific achievement, we must proceed with caution," said Dr. Emily Brown, an environmental 
ethicist. "We need to consider the impact of human activity on Mars, especially when it comes to potential contamination."

The ISRI plans to launch another mission to Mars next year to further investigate the discovery. The new mission, named 
Curiosity 3.0, will be equipped with more advanced technology and is expected to provide more detailed information. 
Curiosity 2.0 will continue to collect data until its operational lifespan expires.

In conclusion, the discovery of water on Mars has opened up a new chapter in space exploration. It has rekindled interest 
in the subject and offers a glimmer of hope for future generations. Only time will tell what other secrets the red planet holds."""

# Extract important keywords
# important_keywords = extract_important_keywords_v3(news_article, num_keywords=5)

# Generate summaries with different detail levels
summary_level_1, summary_level_2 = dynamic_detailed_summarize_premium(news_article, detail_level=1, context_type='article'), \
                                  dynamic_detailed_summarize_premium(news_article, detail_level=2, context_type='article')

important_keywords, summary_level_1, summary_level_2


Top 5 keywords are-:['mars', 'discovery', 'space', 'water', 'isri']
Top 5 keywords are-:['mars', 'discovery', 'space', 'water', 'isri']


(['bookstore', 'wisdom', 'ventured', 'book', 'books'],
 'In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) \nhave found water on Mars. In conclusion, the discovery of water on Mars has opened up a new chapter in space exploration.',
 'In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) \nhave found water on Mars. In conclusion, the discovery of water on Mars has opened up a new chapter in space exploration. The ISRI plans to launch another mission to Mars next year to further investigate the discovery. After months of analysis, the ISRI team confirmed that \nthey have found traces of liquid water. The discovery has significant implications for future space missions, including the possibility of human \ncolonization of Mars.')

In [27]:
# Final example of a story for testing
final_story = """
Once upon a time, in a land filled with magic and wonder, lived a young wizard named Arthur. Arthur was born with a special gift, the ability to speak to animals. However, he lived in a kingdom where magic was forbidden, and the king had ordered all wizards to be captured. Arthur had to keep his abilities a secret to avoid being discovered. One day, Arthur stumbled upon a wounded unicorn in the forest. Using his magic, he healed the unicorn and they became best friends. The unicorn told Arthur about an evil sorcerer who was planning to take over the kingdom. Arthur knew he had to stop him. With the help of the unicorn and other magical creatures, Arthur set out to defeat the sorcerer. After a long and dangerous journey, they finally reached the sorcerer's lair. A battle ensued, and just when it seemed like the sorcerer was going to win, Arthur used a powerful spell to defeat him. The kingdom was saved, and Arthur was hailed as a hero. However, he chose to keep his abilities a secret and returned to his quiet life in the forest, always cherishing the adventure that made him a true wizard.
"""

# Running the summarization function on the final story example
final_story_summary_level1=dynamic_detailed_summarize_premium(final_story, detail_level=1, context_type='story')
final_story_summary_level2 = dynamic_detailed_summarize_premium(final_story, detail_level=2, context_type='story')

final_story_summary_level1,final_story_summary_level2

Top 5 keywords are-:['arthur', 'sorcerer', 'unicorn', 'kingdom', 'magic']
Top 5 keywords are-:['arthur', 'sorcerer', 'unicorn', 'kingdom', 'magic']


('\nOnce upon a time, in a land filled with magic and wonder, lived a young wizard named Arthur. However, he lived in a kingdom where magic was forbidden, and the king had ordered all wizards to be captured. Using his magic, he healed the unicorn and they became best friends.',
 '\nOnce upon a time, in a land filled with magic and wonder, lived a young wizard named Arthur. However, he lived in a kingdom where magic was forbidden, and the king had ordered all wizards to be captured. Using his magic, he healed the unicorn and they became best friends. Arthur knew he had to stop him. Arthur had to keep his abilities a secret to avoid being discovered. The kingdom was saved, and Arthur was hailed as a hero.')

In [28]:
# A long story as a new example
long_story = '''
In a faraway kingdom, there was a brave knight named Sir Cedric. He was known for his courage, wisdom, 
and a heart full of kindness. Despite his numerous victories in battle, Sir Cedric never boasted about his achievements. 
One day, news arrived that a ferocious dragon had been terrorizing a nearby village, burning their crops and houses. 
The king was worried and called upon Sir Cedric to defeat the dragon and save the villagers.

Gathering his armor and sword, Sir Cedric set forth on his perilous journey to the dragon's lair. 
He crossed dark forests, climbed steep mountains, and swam across turbulent rivers. After many days and nights, 
he finally reached the cave where the dragon resided. With his sword at the ready, Sir Cedric stepped into the dark cave, 
only to find the dragon wounded and in pain. Realizing that the dragon was more afraid than ferocious, 
Sir Cedric decided to heal its wounds instead of killing it.

Days passed, and the dragon's health improved. It introduced itself as Eragon and thanked Sir Cedric for his kindness. 
It turned out that Eragon had only attacked the village because hunters from there had tried to steal its eggs. 
Feeling grateful, Eragon promised never to harm anyone again and flew Sir Cedric back to the kingdom.

Upon his return, Sir Cedric narrated his adventure to the king and the villagers. They were amazed at his bravery and wisdom. 
The king rewarded him with a chest full of gold and jewels, but Sir Cedric declined, saying that the real treasure was the friendship 
he had made with Eragon and the lesson he had learned about understanding and kindness.

From that day on, Eragon and the villagers lived in harmony, all thanks to the wise and kind Sir Cedric. 
And they all lived happily ever after.
'''

# Applying the dynamic detailed summarization function

summary_level_1= dynamic_detailed_summarize_premium(long_story, detail_level=1, context_type='story')
summary_level_2 = dynamic_detailed_summarize_premium(long_story, detail_level=2, context_type='story')

summary_level_1, summary_level_2


Top 5 keywords are-:['sir', 'cedric', 'dragon', 'eragon', 'king']
Top 5 keywords are-:['sir', 'cedric', 'dragon', 'eragon', 'king']


('\nIn a faraway kingdom, there was a brave knight named Sir Cedric. Feeling grateful, Eragon promised never to harm anyone again and flew Sir Cedric back to the kingdom. It introduced itself as Eragon and thanked Sir Cedric for his kindness. Upon his return, Sir Cedric narrated his adventure to the king and the villagers.',
 '\nIn a faraway kingdom, there was a brave knight named Sir Cedric. Feeling grateful, Eragon promised never to harm anyone again and flew Sir Cedric back to the kingdom. It introduced itself as Eragon and thanked Sir Cedric for his kindness. Upon his return, Sir Cedric narrated his adventure to the king and the villagers. The king was worried and called upon Sir Cedric to defeat the dragon and save the villagers. Despite his numerous victories in battle, Sir Cedric never boasted about his achievements. From that day on, Eragon and the villagers lived in harmony, all thanks to the wise and kind Sir Cedric. With his sword at the ready, Sir Cedric stepped into the da

* **TF-IDF Vectorization**: Term Frequency-Inverse Document Frequency (TF-IDF) is used to convert text data into numerical form. This is essential for understanding the importance of each word in the document.

* **Cosine Similarity**: This is used to find the similarity between sentences. The idea is to select those sentences that are most similar to the entire document, which are assumed to be the most representative.

* **Sentence Tokenization**: The text is broken down into individual sentences to allow for easier analysis and summary generation.

* **Keyword Extraction**: Keywords are extracted from the text using TF-IDF. These keywords are then used to select important sentences.

* **Context Weighting**: Different weights are given to the number of sentences in the summary based on the type of context (e.g., general, story, report, article).

* **Sentence Selection**: Sentences that contain the important keywords are selected for the final summary.

* **Detail Level Adjustment**: The number of sentences in the final summary can be adjusted based on a detail level parameter.

In [29]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import sent_tokenize
import numpy as np

def extract_important_keywords(input_text, num_keywords=5):
    my_stop_words = ENGLISH_STOP_WORDS.union(['s', 'mr'])
    tfidf_vectorizer = TfidfVectorizer(stop_words=list(my_stop_words), min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    tfidf_values = tfidf_matrix.toarray()[0]
    sorted_indices = tfidf_values.argsort()[::-1]
    top_keywords = feature_names[sorted_indices][:num_keywords]
    print(f'Top {num_keywords} keywords are-:{top_keywords.tolist()}')
    return top_keywords.tolist()

def dynamic_detailed_summarize_premium(input_text, detail_level=1, context_type='general', important_keywords=None):
    if important_keywords is None:
        important_keywords = extract_important_keywords(input_text)
    
    sentences = sent_tokenize(input_text)
    num_sentences = len(sentences)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()
    related_sentences_indices = cosine_similarities.argsort()[:-num_sentences:-1]
    
    important_sentences = [(i, sentences[i]) for i in related_sentences_indices if any(kw in sentences[i].lower() for kw in important_keywords)]
    
    # Context weightage for adjusting the number of sentences based on context_type
    context_weightage = {
        'general': 1,
        'story': 1.5,
        'report': 1.2,
        'article': 1.3  # Adding support for 'article' context
    }
    
    num_output_sentences = max(2, int(detail_level * len(important_sentences) / 5 * context_weightage.get(context_type, 1)))
    
    # Sort the selected sentences based on their original positions to maintain chronological order
    important_sentences = sorted(important_sentences[:num_output_sentences], key=lambda x: x[0])
    selected_sentences = [sentence for index, sentence in important_sentences]
    
    return ' '.join(selected_sentences)



# Testing the function with a sample text
sample_text = """
    Once upon a time, in a faraway kingdom, there lived a king. The king had a beautiful daughter named Rose.
    However, a dragon terrorized the kingdom. The king announced a reward for anyone who could slay the dragon.
    Many knights tried and failed. Then, a young man named Jack decided to attempt it.
    Jack was not a knight, but he had a good heart. He set out on his journey and reached the dragon's lair.
    The dragon was fierce, but Jack was clever. He tricked the dragon and rescued the princess.
    The king was overjoyed and rewarded Jack with half of his kingdom. Jack married Rose, and they lived happily ever after.
"""

dynamic_detailed_summarize_premium(sample_text, detail_level=2, context_type='story')


Top 5 keywords are-:['jack', 'dragon', 'king', 'kingdom', 'lived']


'\n    Once upon a time, in a faraway kingdom, there lived a king. The king had a beautiful daughter named Rose. However, a dragon terrorized the kingdom. The king announced a reward for anyone who could slay the dragon. The king was overjoyed and rewarded Jack with half of his kingdom. Jack married Rose, and they lived happily ever after.'

# Comparing Two Technique of Important Keyword 

## 1st-: Term Frequency-Inverse Document Frequency (TF-IDF) algorithm

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import numpy as np



# Initialize Spacy for NER
nlp = spacy.load("en_core_web_sm")

# Function to extract important named entities
def extract_named_entities(input_text):
    doc = nlp(input_text)
    named_entities = [ent.text.lower() for ent in doc.ents]
    return named_entities

# Function to extract important key phrases
def extract_key_phrases(input_text, num_key_phrases=5):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    tfidf_values = tfidf_matrix.toarray()[0]
    sorted_indices = tfidf_values.argsort()[::-1]
    top_key_phrases = feature_names[sorted_indices][:num_key_phrases]
    return top_key_phrases.tolist()

# Function to extract important keywords, now including named entities and key phrases
def extract_important_keywords(input_text, num_keywords=5):
    # Extract named entities
    named_entities = extract_named_entities(input_text)
    
    # Extract key phrases
    key_phrases = extract_key_phrases(input_text, num_keywords)
    
    # Combine named entities and key phrases and take the unique ones
    important_keywords = list(set(named_entities + key_phrases))
    
    # If more than num_keywords, truncate the list
    if len(important_keywords) > num_keywords:
        important_keywords = important_keywords[:num_keywords]
    
    print(f'Top {num_keywords} keywords are-:{important_keywords}')
    return important_keywords

# Sample text for testing
sample_text = """In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) 
have found water on Mars. After months of analysis, the ISRI team confirmed that 
they have found traces of liquid water. The discovery has significant implications for future space missions, 
including the possibility of human colonization of Mars."""

# Test the function
extract_important_keywords(sample_text, num_keywords=5)


Top 5 keywords are-:['isri', 'months', 'water', 'space', 'the international space research institute']


['isri',
 'months',
 'water',
 'space',
 'the international space research institute']

## Named Entity Recognition

In [31]:
# Importing the required packages for Named Entity Recognition (NER)
import spacy

def extract_named_entities(input_text, num_entities=5):
    # Load the English NLP model from spaCy
    nlp = spacy.load("en_core_web_sm")
    
    # Process the input text using spaCy NLP pipeline
    doc = nlp(input_text)
    
    # Create a dictionary to hold entities and their counts
    entity_counts = {}
    
    # Loop through all the named entities in the processed document
    for ent in doc.ents:
        # Only consider entities that are not dates, numbers, or percentages
        if ent.label_ not in ["DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"]:
            entity_counts[ent.text.lower()] = entity_counts.get(ent.text.lower(), 0) + 1
    
    # Sort the entities by their counts in descending order
    sorted_entities = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Select the top num_entities
    top_entities = [entity[0] for entity in sorted_entities[:num_entities]]
    
    print(f'Top {num_entities} named entities are-:{top_entities}')
    return top_entities

# Test the function with a sample text
sample_text = '''
In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) 
have found water on Mars. The ISRI plans to launch another mission to Mars next year to further investigate the discovery. 
After months of analysis, the ISRI team confirmed that they have found traces of liquid water. 
The discovery has significant implications for future space missions, including the possibility of human 
colonization of Mars.
'''

# Extract top 5 named entities
extract_named_entities(sample_text)


Top 5 named entities are-:['mars', 'the international space research institute']


['mars', 'the international space research institute']

# Detailed-Dynamic Context-Aware Summarization
## MEthods Names-:
* **TF-IDF Vectorization**: Used to convert text into a numerical format that can be easily analyzed. Specifically used for keyword extraction.
* **Cosine Similarity**: To find the similarity between sentences.
* **Named Entity Recognition (NER)**: To extract important names, places, and other entities from the text.
* **Sentence Tokenization**: To break down the text into individual sentences.
* **Stop Words Removal**: To ignore commonly occurring words that don't carry significant meaning.
* **Dynamic Detail Levels**: To adjust the length and detail of the summary.
* **Context-based Weightage**: To adjust the number of sentences in the summary based on the type of text (e.g., story, report, article).

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize
import numpy as np

# Function to extract important keywords using TF-IDF
def extract_important_keywords(input_text, num_keywords=5):
    my_stop_words = list(ENGLISH_STOP_WORDS)
    tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words, min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    tfidf_values = tfidf_matrix.toarray()[0]
    sorted_indices = tfidf_values.argsort()[::-1]
    top_keywords = feature_names[sorted_indices][:num_keywords]
    return top_keywords.tolist()

# Function to extract named entities using SpaCy
def extract_named_entities(input_text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(input_text)
    named_entities = [ent.text.lower() for ent in doc.ents]
    named_entities=named_entities
    return list(set(named_entities))

# Function to generate dynamic detailed summaries
# Let's make a modification to the sentence ordering in the summary generation function.
def dynamic_detailed_summarize_premium(input_text, detail_level=1, context_type='general', important_keywords=None, important_entities=None):
    if important_keywords is None:
        important_keywords = extract_important_keywords(input_text)
    
    if important_entities is None:
        important_entities = extract_named_entities(input_text)


    print(f'Keywords are-: {important_keywords}')
    print(f'Entities are-: {important_entities}')    
    # Combine keywords and named entities for a more comprehensive set of important words
    important_words = set(important_keywords + important_entities)
    
    sentences = sent_tokenize(input_text)
    num_sentences = len(sentences)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()
    related_sentences_indices = cosine_similarities.argsort()[:-num_sentences:-1]

    important_sentences = [sentences[i] for i in related_sentences_indices if any(kw.lower() in sentences[i].lower() for kw in important_words)]
    
    # Context weightage for adjusting the number of sentences based on context_type
    context_weightage = {
        'general': 1,
        'story': 1.5,
        'report': 1.2,
        'article': 1.3  # Adding support for 'article' context
    }
    
    num_output_sentences = max(2, int(detail_level * len(important_sentences) / 5 * context_weightage.get(context_type, 1)))
    
    # Sort the sentences based on their original order to maintain chronological sequence
    selected_sentences = sorted(important_sentences[:num_output_sentences], key=lambda x: sentences.index(x))
    
    return ' '.join(selected_sentences)


# Example usage
example_text = '''
In a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) 
have found water on Mars. After months of analysis, the ISRI team confirmed that 
they have found traces of liquid water. The discovery has significant implications for future space missions, including the possibility of human 
colonization of Mars. In conclusion, the discovery of water on Mars has opened up a new chapter in space exploration. 
The ISRI plans to launch another mission to Mars next year to further investigate the discovery.
'''

# Extract important keywords and named entities
# important_keywords = extract_important_keywords(example_text)
# named_entities = extract_named_entities(example_text)

# Generate summary
dynamic_detailed_summarize_premium(example_text, detail_level=2, context_type='report')



Keywords are-: ['mars', 'discovery', 'isri', 'water', 'space']
Entities are-: ['mars', 'months', 'the international space research institute', 'next year']


'\nIn a groundbreaking discovery, scientists at the International Space Research Institute (ISRI) \nhave found water on Mars. In conclusion, the discovery of water on Mars has opened up a new chapter in space exploration.'

In [33]:
# Example of a long story text
long_story_text = """
Once upon a time in a small village, there lived a young boy named Tim. Tim was an adventurous kid who was always looking for something exciting to do. 
One day, he heard about a magical treasure hidden deep within the Haunted Forest, a place most villagers avoided due to the frightening stories they'd heard. 
Fueled by curiosity and the allure of the treasure, Tim decided to embark on a journey to find it.

Before setting out, Tim consulted the village's oldest and wisest resident, Old Man Jenkins. Jenkins had lived in the village for as long as anyone could remember and was a treasure trove of knowledge and folklore. 
"Ah, the Haunted Forest," mused Jenkins, "Many have entered, but few have returned. If you wish to go, you must be prepared. Take this enchanted compass; it will guide you to the treasure but beware of the guardian spirit that protects it."

Armed with his new compass and a bag full of supplies, Tim set out for the Haunted Forest. The journey was perilous, filled with dangerous animals, treacherous terrain, and all sorts of mystical barriers. 
But Tim was relentless. Guided by the enchanted compass, he finally reached the heart of the forest where the treasure was said to be hidden. As he took his final steps towards it, the guardian spirit appeared.

"Who dares to seek the treasure of the Haunted Forest?" boomed the spirit. Tim, although frightened, stood his ground. 
"I am Tim, and I seek the magical treasure," he replied. The spirit looked into Tim's eyes and saw the purity of his intentions. 
"Very well," said the spirit, "You may take the treasure, but remember, it comes with a responsibility to use it wisely."

Tim took the treasure, a chest filled with gold and magical artifacts, and returned to his village. He was hailed as a hero and went on to use the treasure to improve the lives of all the villagers. 
He built a school, a hospital, and even brought electricity to the village. And as for the guardian spirit, it continued to protect the forest but was never again seen by the villagers. 
Tim grew up to be a wise and generous leader, always remembering the lessons he had learned on his adventure.

And so, Tim and his fellow villagers lived happily ever after. The end.
"""

dynamic_detailed_summarize_premium(long_story_text,1,'story')

Keywords are-: ['tim', 'treasure', 'spirit', 'forest', 'village']
Entities are-: ['ah, the haunted forest', 'tim', 'the haunted forest', 'jenkins', 'one day', 'man jenkins']


"\nOnce upon a time in a small village, there lived a young boy named Tim. Before setting out, Tim consulted the village's oldest and wisest resident, Old Man Jenkins. Jenkins had lived in the village for as long as anyone could remember and was a treasure trove of knowledge and folklore. But Tim was relentless. Tim took the treasure, a chest filled with gold and magical artifacts, and returned to his village. And so, Tim and his fellow villagers lived happily ever after."

In [34]:
article4="""Character.AI, the a16z-backed AI chatbot startup from ex-Google AI researchers, is out today with a new feature for its subscribers. The chatbot platform, which offers customizable AI companions with distinct personalities and tools to make your own, is now offering a group chat experience where users and their friends can chat with multiple AI characters at once.

The Character Group Chat feature, as it’s called, allows users to create a group chat with their favorite AI characters only or it can feature a mix of both humans and AI companions, the company says. The idea is that users will be able to create social connections with friends, or share ideas and collaborate in real-time, as in any other group chat experience, but with their AI companions now in the mix.

The company suggests users could try out having AI scientists and thinkers chat together, like  Albert Einstein, Marie Curie, Nikola Tesla, and Stephen Hawking, for example, or create a group chat with mythological gods like Zeus, Hades, and Poseidon.

For more practical use cases, you might start a group chat with friends around a topic or theme — like travel, gaming, book clubs, or role-playing — then invite an AI companion to help facilitate and augment those conversations.

The idea of adding AI chatbots into a group chat is not unique to Character.AI. Snapchat’s My AI chatbot can be added into group chats with the command @myai, while Meta recently introduced the ability to call up a host of new AI-powered bots across its apps, including WhatsApp, Messenger and Instagram’s DMs, including those that are based on celebrities like Mr. Beast, Paris Hilton, Tom Brady, Charli D’Amelio, Snoop Dog, and others. The latter announcement, made at Meta’s Connect conference in late September, was a potential threat to Character.AI which raised a whopping $150 million in Series A funding earlier this year for its concepts around AI companions.

However, Character.AI’s new group chat experience won’t be offered for free. Instead, the feature is first being made available to the c.ai+ subscribers in order to gain feedback and make improvements. C.ai+ is the startup’s $9.99 per month subscription plan that offers the ability to skip waiting rooms and access to faster message generation as well as an exclusive community channel for feedback and support, among other things. The company says it will later open up to feature to the general public.

At launch, Character Group Chat is also only available on the Character.AI mobile app on iOS and Android but will later roll out to the web.

The company’s app initially topped half a million installs in its first six days and is said to be catching up with ChatGPT in the U.S. Third-party data from market intelligence provider data.ai indicates the app has close to 30 million monthly active users globally, and around 7 million in the U.S. The firm also estimates its lifetime gross in-app purchase revenue is $1.3 million, but c.ai+ is sold on the web so this is not a comprehensive look at its overall revenue."""
dynamic_detailed_summarize_premium(article4,1,'article')

Keywords are-: ['ai', 'chat', 'group', 'character', 'million']
Entities are-: ['character', 'paris hilton', 'first', '$150 million', 'ai', 'whatsapp', 'charli d’amelio', 'albert einstein', 'tom brady', 'android', 'meta', 'beast', 'meta’s connect', 'earlier this year', 'the character group chat', 'monthly', 'poseidon', 'today', 'character group chat', 'zeus', 'its first six days', 'ai chatbot', 'half a million', '30 million', 'nikola tesla', '9.99', 'u.s.', 'around 7 million', '$1.3 million', 'series', 'stephen hawking', 'marie curie', 'snoop dog', 'instagram', 'late september', 'third', 'messenger']


'Character.AI, the a16z-backed AI chatbot startup from ex-Google AI researchers, is out today with a new feature for its subscribers. The Character Group Chat feature, as it’s called, allows users to create a group chat with their favorite AI characters only or it can feature a mix of both humans and AI companions, the company says. However, Character.AI’s new group chat experience won’t be offered for free. Instead, the feature is first being made available to the c.ai+ subscribers in order to gain feedback and make improvements.'

In [35]:
story2="""In the heart of a bustling city, there lived a young woman named Emma. Emma was a skilled violinist who played her music on the street corner every day. Her melodies were like magic, and they filled the air with joy.

But Emma had a secret. When she played her violin, the music had the power to heal. People who stopped to listen found their troubles and worries melting away. Emma's music could mend broken hearts, ease pain, and bring smiles to the faces of those who had forgotten how to smile.

One chilly evening, as Emma played her violin, a homeless man named Samuel sat on the cold pavement, shivering and hungry. He had lost everything and felt completely alone. But when he heard Emma's music, he was drawn to her, as if an invisible force was guiding him.

He sat beside Emma, tears streaming down his face. Her music wrapped around him like a warm blanket, and his despair began to fade. Emma played with all her heart, pouring love and compassion into her melodies.

As the night wore on, the street filled with people who had been touched by Emma's music. They came with food, warm blankets, and kind words. Samuel, who had been a stranger, now had friends who cared for him.

Over time, Emma's reputation as the "Healing Violinist" grew. People came from far and wide to listen to her music and experience its soothing power. She used her talent to bring comfort to those in need, to heal the broken, and to remind everyone that even in the midst of a bustling city, there was room for kindness and compassion.

And so, in the heart of the bustling city, Emma's violin continued to play, a beacon of hope and healing for all who had the privilege to listen.

"""
dynamic_detailed_summarize_premium(story2,2,'story')

Keywords are-: ['emma', 'music', 'played', 'listen', 'violin']
Entities are-: ['one', 'every day', 'emma', 'the night', 'the "healing violinist', 'samuel']


'In the heart of a bustling city, there lived a young woman named Emma. Emma was a skilled violinist who played her music on the street corner every day. But Emma had a secret. One chilly evening, as Emma played her violin, a homeless man named Samuel sat on the cold pavement, shivering and hungry. But when he heard Emma\'s music, he was drawn to her, as if an invisible force was guiding him. He sat beside Emma, tears streaming down his face. Emma played with all her heart, pouring love and compassion into her melodies. Over time, Emma\'s reputation as the "Healing Violinist" grew. She used her talent to bring comfort to those in need, to heal the broken, and to remind everyone that even in the midst of a bustling city, there was room for kindness and compassion. And so, in the heart of the bustling city, Emma\'s violin continued to play, a beacon of hope and healing for all who had the privilege to listen.'

In [36]:
summarized_story="""'In the heart of a bustling city, there lived a young woman named Emma. Emma was a skilled violinist who played her music on the street corner every day. But Emma had a secret. One chilly evening, as Emma played her violin, a homeless man named Samuel sat on the cold pavement, shivering and hungry. But when he heard Emma\'s music, he was drawn to her, as if an invisible force was guiding him. He sat beside Emma, tears streaming down his face. Emma played with all her heart, pouring love and compassion into her melodies. Over time, Emma\'s reputation as the "Healing Violinist" grew. She used her talent to bring comfort to those in need, to heal the broken, and to remind everyone that even in the midst of a bustling city, there was room for kindness and compassion. And so, in the heart of the bustling city, Emma\'s violin continued to play, a beacon of hope and healing for all who had the privilege to listen.'"""
dynamic_detailed_summarize_premium(summarized_story,1,'story')

Keywords are-: ['emma', 'bustling', 'heart', 'city', 'played']
Entities are-: ['one', 'every day', 'emma', 'the "healing violinist', 'samuel']


"'In the heart of a bustling city, there lived a young woman named Emma. And so, in the heart of the bustling city, Emma's violin continued to play, a beacon of hope and healing for all who had the privilege to listen.'"

In [37]:
# Generate a longer story example
long_story_example = """
Once upon a time in a kingdom far, far away, there was a wise and just king named Arthur. King Arthur had a daughter, 
Princess Fiona, who was the jewel of his eye. The kingdom was peaceful and prosperous, but this peace was shattered when a 
dragon started terrorizing the village. The dragon would fly in from its mountain lair and set houses on fire, steal livestock, 
and frighten the children. The villagers were terrified and turned to King Arthur for help.

King Arthur was deeply concerned and announced a reward for anyone who could slay the dragon. Knights from all around the kingdom 
and even from distant lands came to try their luck, but none succeeded. The dragon was too strong and cunning for them.

One day, a humble farmer named Jack decided to take on the challenge. Jack was poor but very clever. He had read in ancient 
texts about a magical sword that was designed to kill dragons. This sword was hidden in a cave guarded by mystical creatures. 
Jack embarked on a perilous journey to find the sword. He fought off wolves, crossed treacherous terrains, and finally reached the 
cave. With his wit and bravery, he defeated the mystical creatures and acquired the magical sword.

Armed with the magical sword, Jack returned to the kingdom and faced the dragon. After a fierce battle, Jack managed to slay the 
dragon and saved the kingdom. The villagers celebrated his bravery and King Arthur rewarded him with half of his kingdom and 
the hand of Princess Fiona in marriage.

Jack married Fiona and became the new king after Arthur. Under his rule, the kingdom was peaceful, prosperous, and free from any 
danger. Jack and Fiona lived happily ever after, and so did their subjects. The end.
"""

# Extract important keywords and named entities, and generate the summary
dynamic_detailed_summarize_premium(long_story_example, detail_level=1, context_type='story')



Keywords are-: ['jack', 'kingdom', 'king', 'arthur', 'dragon']
Entities are-: ['half', 'fiona', 'knights', 'arthur', 'king arthur', 'one day', 'jack']


'\nOnce upon a time in a kingdom far, far away, there was a wise and just king named Arthur. King Arthur had a daughter, \nPrincess Fiona, who was the jewel of his eye. The villagers were terrified and turned to King Arthur for help. The villagers celebrated his bravery and King Arthur rewarded him with half of his kingdom and \nthe hand of Princess Fiona in marriage. Jack married Fiona and became the new king after Arthur.'

In [38]:
article5="""Artificial Intelligence (AI) is rapidly transforming industries across the globe. From self-driving cars to virtual personal assistants, AI is making significant advancements that promise to improve efficiency and productivity. However, as AI technology continues to evolve, it also raises concerns about its impact on the labor market.

AI has the potential to automate various tasks and processes, leading to increased efficiency and reduced labor costs for businesses. For example, in the manufacturing sector, robots and AI-powered machines can perform repetitive and physically demanding tasks more accurately and tirelessly than human workers. This reduces the need for manual labor and can lead to cost savings for companies.

In the service industry, virtual assistants and chatbots are increasingly replacing human customer service representatives. These AI systems can handle customer inquiries, provide support, and even process orders 24/7, eliminating the need for a large workforce. This automation can improve response times and reduce operational costs.

In the financial sector, AI algorithms can analyze vast amounts of data to make investment decisions, manage portfolios, and detect fraudulent activities. This not only enhances the accuracy of financial operations but also reduces the need for human financial analysts.

However, the increasing automation of jobs through AI can also lead to job displacement and concerns about unemployment. While some argue that AI will create new jobs in AI development and maintenance, there are valid worries about the workers who may not have the skills or resources to transition into these new roles.

To address these concerns, governments and organizations must invest in workforce development and retraining programs to help workers acquire the skills necessary for the jobs of the future. Additionally, there is a growing movement to establish universal basic income (UBI) programs to provide financial support to individuals who lose their jobs due to AI and automation.

Despite these challenges, AI also presents new opportunities for innovation and growth. As AI continues to advance, it opens up new avenues for businesses to develop novel products and services. For example, AI-powered healthcare applications can improve patient care and diagnostics. AI-driven transportation systems can enhance the safety and efficiency of travel.

In conclusion, the impact of AI on the labor market is multifaceted. While it brings the promise of increased efficiency and reduced costs for businesses, it also poses challenges related to job displacement. To harness the full potential of AI, it is crucial to invest in workforce development and explore new economic models that ensure the well-being of all individuals in an AI-driven world.

"""
dynamic_detailed_summarize_premium(article5,1,'article')

Keywords are-: ['ai', 'new', 'jobs', 'financial', 'efficiency']
Entities are-: ['artificial intelligence (ai', 'ai', '24/7']


'Artificial Intelligence (AI) is rapidly transforming industries across the globe. While some argue that AI will create new jobs in AI development and maintenance, there are valid worries about the workers who may not have the skills or resources to transition into these new roles. In conclusion, the impact of AI on the labor market is multifaceted. To harness the full potential of AI, it is crucial to invest in workforce development and explore new economic models that ensure the well-being of all individuals in an AI-driven world.'

In [39]:
text="""A security researcher says a bug on an Indian state government website inadvertently revealed documents containing residents’ Aadhaar numbers, identity cards, and copies of their fingerprints.

The bug was fixed last week after the security researcher disclosed the bug to local authorities.

Sourajeet Majumder found the bug in the West Bengal government’s e-District web portal that allows state residents to access government services online, like obtaining birth and death certificates and building applications. Majumder said the website bug meant it was possible to obtain land deeds, which contain records about the owners of a piece of land, from the e-District website by guessing sequential deed application numbers.

Application identification numbers are unique 16-digit numbers issued by the state government when a local resident applies for a digital copy of a deed. Not every application identification number was valid. Using publicly available tools like Burp Suite to analyze the network traffic in and out of the website meant that Majumder could cycle through entire lists of sequential application numbers and use the responses from the server to determine if an application identification number was valid.

With access to an application identification number, anyone with a login to the e-District system could access a copy of a land deed. Two land deed records seen by TechCrunch contain the names of the individuals involved with the deed, their photographs, and their full set of fingerprints from both hands. It’s not uncommon to see multiple individuals on a single deed.

The deeds also contain the individuals’ government-issued identity documents, including their confidential Aadhaar numbers, which every citizen is assigned as part of India’s national identity and biometric database. Aadhaar numbers are required for accessing banking, cell phone plans, and many government services.

Majumder reported the website vulnerability to India’s computer emergency response team, known as CERT-In, and the West Bengal government, fearing that the vulnerability could be misused for identity fraud. The bug was fixed soon after.

It’s not known if anyone else other than Majumder discovered the bug. Representatives for the West Bengal government and CERT-In did not return requests for comment. The West Bengal government’s e-District website says it has processed more than 17 million applications to date, though it’s not known how many relate to land deeds.

Local media reports a recent rise in fraud linked to the alleged theft of biometric information, which criminals are said to be using to empty bank accounts."""

In [40]:
dynamic_detailed_summarize_premium(text,1,'article')

Keywords are-: ['government', 'numbers', 'bug', 'deed', 'website']
Entities are-: ['16', 'india', 'two', 'burp suite', 'more than 17 million', 'west bengal', 'techcrunch', 'last week', 'aadhaar', 'indian']


'A security researcher says a bug on an Indian state government website inadvertently revealed documents containing residents’ Aadhaar numbers, identity cards, and copies of their fingerprints. The bug was fixed last week after the security researcher disclosed the bug to local authorities. Sourajeet Majumder found the bug in the West Bengal government’s e-District web portal that allows state residents to access government services online, like obtaining birth and death certificates and building applications. The deeds also contain the individuals’ government-issued identity documents, including their confidential Aadhaar numbers, which every citizen is assigned as part of India’s national identity and biometric database.'

In [78]:
article6="""Gaza, despite being one of the most economically challenged regions in the world, has ironically always been a tech hub — not only for Palestine and Palestinians, but for the world: international companies have, for many years, sought out a presence there to collaborate both with with talented tech freelancers, and the startups which gradually emerged from the region. For examples, according to sources who helped build those bridges, Nvidia, famed for it’s role in the new AI boom, has been working with at least 100 engineers from the region for years.
Since at least 2008 TechCrunch been covering technology companies out of Palestine, some serving their direct audience, some serving the tech world internationally. Silicon Valley had taken an increasing interest in Palestine as a tech hub, but like the ecosystem itself, it’s nascent: to date, those working in the region estimate that much as $10 million has been invested in the Palestinian tech ecosystem.
Notably, in 2017, Salesforce founder and CEO Marc Benioff joined Silicon Valley luminaries in backing the first ever coding academy to be created in Gaza.
Gaza Sky Geeks, an Alphabet-backed initiative based in Gaza that provides pre-seed investments, training and technology resources to Palestine’s Gazan population, has been a beacon of entrepreneurship in the region.
All of that is now, effectively, gone, like the buildings in Gaza itself.
Israel is currently retaliating militarily against the attacks on its people, on its soil, and the hostages subsequently taken by Hamas — the ruling organisation in Gaza that kidnapped at least 150 people and took them into Gaza during brutal attacks on Israel at the weekend that killed 1,300 people.
That strategy has seen it pummelling the ‘Gaza Strip’ with bombs to eradicate it of Hamas and to get its hostages back. Over 1,500 people in Palestine so far have been killed as a result. The tech industry in Israel — the country’s biggest export, and its biggest single contributor to GDP — is also taking a big knock (read about that here), but the impact on the smaller and more fragile ecosystem in Gaza has been, inevitably, significantly more serious. The physical, economic and societal destruction resulting from that leaves any future for the tech industry there in doubt.
Quite simply, there is no escaping the consequences of the war for anyone, let alone tech workers.
“What is happening to tech in Gaza is that Israel is crunching it. Obliterating it,” one source, inside the territory, told TechCrunch.
Israel has now amassed soldiers near the north of Gaza, ahead of an expected ground offensive into the densely populated enclave. About 1.1 million people living in northern areas have been told to leave in the next day. The UN has warned of “devastating humanitarian consequences” from these latest moves. A total blockade on the territory is being enforced with fuel, food and water running out. Israel says it won’t lift the restrictions unless Hamas frees all hostages.
Speaking to Ryan Sturgill, an American national and former head of the Gaza Sky Geeks accelerator run by sponsor Mercy Corps, and NGO aid organisation, the situation on the ground appears dire, after waves of shelling by the Israeli military.
“The area around the the Mercy Corps building, which housed Gaza Sky Geeks, has been levelled. The structure is standing but blown out. The front of it is sort of ripped off,” he said.
Gaza Sky Geeks (GSG) is the largest tech-hub in Palestine, providing a wide range of tech training at scale. In 2022, 5,000 coders and developers from across the West Bank and Gaza graduated from the programme.
Video evidence (pictured above) posted on Linkedin shows a blown-out building with the Mercy Corps sign.
“Who knows what’s going to happen. The offices are destroyed, the fibre lines are destroyed. The universities are destroyed. Three main universities in Gaza that produce all the computer science grads are levelled. I don’t even know if people will be ever be able to go back to Northern Gaza after what’s happening today.The educational institutions that are there are gone,” Sturgill added.
He had been helping Palestinian tech startups raise capital in the West Bank and Gaza since January.
“Until now, there had been a pretty significant growth. A lot of companies in Saudi Arabia have been setting up back offices [in Palestine] for development for all sorts of new companies and even apps that are that are now growing in the Gulf, because Saudi has been growing so quickly on the tech front. Nvidia, and other international companies, has outsourcing operations in Palestine. Apple has outsourcing operations, Microsoft has R&D, and they would even like to see those expand. There’s companies that had 200 developers sitting in offices in Ramallah,” he said.
“I’ve talked to all of the heads of these different offices, most of them are in in Israel. They are very positive people who want to try to support the tech industry there and those efforts have been working well and growing,” he added.
Indeed, one of the main Palestinian VC funds, Ibtikar, had recently raised its second fund of $30 million.
High growth companies emerging from Palestine include Menalytics (data analytics, invested in by Flat 6 labs); Olivery (last mile logistics, Flat6Labs and Ibtikar Fund); Coretava (employee and customer loyalty); and Sellenvo (an Amazon fulfillment partner).
Sturgill said that as well as the intensely difficult conditions in Gaza, which is being hit by Israeli missiles, the situation in Ramallah is “super tense. I feel like the situation is going to get significantly worse there over the coming weeks.”
Iliana Montauk, is co-Founder and CEO of Manara — a social impact startup funded by Y Combinator, Seedcamp, Reid Hoffman, Eric Ries, Marc Benioff, Paul Graham, and Jessica Livingston, among others — told Techcrunch via email that connectivity has decreased significantly in the past 24 hours.
“Though Gaza has been bombed many times before, this time is completely different for the tech sector for several reasons. Electricity was cut off to the entire [Gaza] strip. A significant amount of infrastructure has been bombed (including both ISPs and many tall apartment buildings that hold cell phone towers). Entire middle-class neighborhoods are being destroyed.”
She said in the past if an entire neighborhood got destroyed, it was usually one bordering Israel and a poorer area, thus less impacting the tech sector.
“The tech sector is almost completely unable to function in Gaza right now,” she said. “Most people are in too much danger to be able to work; some have evacuated three times in the past 24 hours, moving from friend’s house to family house, because each neighborhood they end up in is the next one being bombed. They usually receive warnings to evacuate their homes 10 minutes before a bombing, so they don’t sleep and monitor the situation constantly ready to evacuate in a minute’s notice.”
“Most people have lost cell phone connections and internet access completely, or have some access to 2G only on their cell phones.. Electricity is no longer being provided even for a few hours a day now, and people are running out of gas for their generators,” she added.
Manara has around 100 software engineers in Gaza, some working remotely for tech companies in Silicon Valley/Europe.
Montauk said one software engineer who works at Upwork disappeared for several days, until being found alive.
Dalia Awad, whose Medium post about getting into Google from Gaza went viral in 2021 (it was at one point #1 on Hacker News and tweeted by Paul Graham), returned to Gaza after her internships at Google and Datadog to graduate from university. She had a full-time job offer at Datadog in Paris but decided to stay home in Gaza and look for a remote job so she could be close to family.
On Tuesday she wrote to Montauk saying: “Tonight was the worst night ever. My family and I are good, thankfully. The bombing was everywhere and we couldn’t know where it was because there was no internet. Many of my friends lost their houses in the Rimal area.. There is no wifi internet, we connect to the cellular data on our phones but it’s only 2G and it connects for a few minutes then cuts off. We can only send Whatsapp messages. So we can’t really read news on social media. In the morning we saw these videos from our friends who shared it on whatsapp but it takes FOREVER to download a video of a few seconds.”
Montauk said Awad had not responded to her in the past day.
Mai Temraz, Manara’s first employee, is based in San Bernardino, California. Her family live in Gaza City. They narrowly escaped a bombing (she posted [Content warning] a video on Instagram of them bleeding). She said: “My family barely survived an attack on a building next to them in Gaza. They ask people to leave, WHERE?? No one is save [sic.] anywhere in Gaza.”
Montauk, a former director of Gaza Gaza Sky Geeks, said: “Before this escalation, the Gaza tech scene was growing. I was just in Riyadh and met companies that hire entire software development teams in Gaza. Upwork and other Silicon Valley companies are now hiring software engineers remotely from Gaza. In addition, some had left to work abroad at companies like Google, Amazon, Qualtrics, etc. Last time I was in Gaza a year ago, almost everyone I talked to asked me how they could get a job and leave Gaza. They were worried about more bombings and wanted to bring their children up in a place without such a high risk. These people just want to live normal lives.”
Those who are living in the West Bank say the activity in Gaza has had an inevitable impact.
“For a Palestinian young woman like me living in the West Bank, I can confirm that there has been a noticeable freeze in terms of activity,” said Leen Abubaker of Flow Accelerator and Cofounder at Sawaed19. “Tech companies are either operating on a very limited scale, with employees struggling to reach their offices in the West Bank due to unsafe roads blocked by Israeli occupation forces and settlers, or they have been forced out of business entirely in Gaza.”
She added that a number of building in Gaza key to the tech industry there, such as Burj Al-Wattan, had been destroyed by Israeli airstrikes and that the tech industry is not the first priority people in the urgent situation. “How can you possibly detach from the distressing reality and cling to the remaining shreds of hope for your business?”
Mohammad Alnobani is a Palestinian founder of The Middle Frame, an Arab stock image platform powered by AI tools, aiming to shatter stereotypes about the Arab world through images and reducing bias in AI.
He told me he was on his way back from the One Young World Summit in Belfast, speaking about peace and reconciliation, and about to reach the borders to cross to Palestine to get back to his family, when the war broke out.
“The borders closed down and I had to turn around and go back to Jordan,” he said. “I am still there, continuously checking on my family in Jerusalem, and trying to reach out to my connections in Gaza.” His co-founder, Raya Fatayer, is in Ramallah, staying at home with her baby and husband, not able to travel.
“Our fellow entrepreneurs in Gaza had their homes demolished by air strikes, some we can’t even reach anymore since the electricity is out and they have no power,” he said. “Dealing with the situation while trying our best to move forward with our work is a daily challenge.”
He said this outbreak of hosilities with Israel is clearly different: “Before, every time Gaza faced air strikes, we knew that certain areas were almost safe. Clearly today, nobody is safe.”
"""


In [79]:
dynamic_detailed_summarize_premium(article6,1,'article')

Keywords are-: ['gaza', 'tech', 'people', 'said', 'companies']
Entities are-: ['leen abubaker', 'manara', 'a few hours', 'the morning', 'around 100', 'years', 'silicon valley', 'silicon valley/europe', 'gazan', 'hacker news', 'linkedin', 'apple', 'jerusalem', 'instagram of them', '1,300', 'at least 150', '2', 'about 1.1 million', 'eric ries', 'nvidia', 'northern gaza', 'the middle frame', 'reid hoffman', 'first', 'the coming weeks', 'google and datadog', 'riyadh', 'ai', 'whatsapp', 'techcrunch', 'at least 2008', '1,500', 'the gaza sky geeks', 'forever', '200', 'daily', 'today', 'raya fatayer', 'jordan', 'january', 'tuesday', 'mai temraz', 'one', 'ibtikar fund', 'sellenvo', 'three', 'dalia awad', 'datadog', 'the west bank', 'one point #1', 'arab', 'marc benioff', 'mercy corps', 'a year ago', 'olivery', 'paul graham', 'sky geeks', 'gsg', 'seedcamp', '10 minutes', 'burj al-wattan', 'several days', 'belfast', '2022', 'ramallah', 'tonight', 'coretava', 'gaza city', 'gaza', 'the next day', '

'Gaza, despite being one of the most economically challenged regions in the world, has ironically always been a tech hub — not only for Palestine and Palestinians, but for the world: international companies have, for many years, sought out a presence there to collaborate both with with talented tech freelancers, and the startups which gradually emerged from the region. For examples, according to sources who helped build those bridges, Nvidia, famed for it’s role in the new AI boom, has been working with at least 100 engineers from the region for years. Since at least 2008 TechCrunch been covering technology companies out of Palestine, some serving their direct audience, some serving the tech world internationally. Silicon Valley had taken an increasing interest in Palestine as a tech hub, but like the ecosystem itself, it’s nascent: to date, those working in the region estimate that much as $10 million has been invested in the Palestinian tech ecosystem. Gaza Sky Geeks, an Alphabet-bac

In [46]:
story3="""Once upon a time, there were three little pigs. Each pig was given some money by their mother and told to go out into the world, build a house, and make a life for themselves.

The first little pig was not very wise and decided to build a house out of straw because it was quick and easy. He finished his straw house in no time and spent the rest of the day playing. But soon, a big, bad wolf came along and said, "Little pig, little pig, let me in!" The pig refused, so the wolf huffed and puffed and blew the straw house down. The first little pig had to run to his brother's house for safety.

The second little pig was a bit smarter. He chose to build his house out of sticks. This house was sturdier than the straw house, but the wolf came along and said, "Little pig, little pig, let me in!" The pig also refused, so the wolf huffed and puffed and blew the stick house down. The two little pigs ran to their older brother's house for safety.

The third little pig was the wisest of them all. He decided to build a strong house out of bricks. It took him longer, but he knew it would be worth it. When the wolf came to the brick house and said, "Little pig, little pig, let me in!" the third little pig refused just like his brothers. The wolf huffed and puffed, but he couldn't blow the brick house down.

Frustrated, the wolf tried to climb down the chimney to catch the little pig, but the clever pig had a pot of boiling water waiting for him. The wolf fell into the pot and ran away, never to bother the three little pigs again.

The three little pigs were safe in their brick house, and they learned the importance of hard work, determination, and making wise choices.

And they all lived happily ever after in their sturdy brick house.

"""


In [47]:
dynamic_detailed_summarize_premium(story3,1,'story')

Keywords are-: ['pig', 'little', 'house', 'wolf', 'straw']
Entities are-: ['first', 'two', 'second', 'three', 'wolf', 'third']


'Once upon a time, there were three little pigs. He finished his straw house in no time and spent the rest of the day playing. The two little pigs ran to their older brother\'s house for safety. When the wolf came to the brick house and said, "Little pig, little pig, let me in!" The wolf fell into the pot and ran away, never to bother the three little pigs again. The three little pigs were safe in their brick house, and they learned the importance of hard work, determination, and making wise choices.'

In [48]:
story4="""Once upon a time in a quiet village, there lived a young girl named Lily. She was known for her insatiable curiosity and adventurous spirit. Lily spent most of her days exploring the nearby forest, searching for hidden treasures, and befriending the creatures that called the woods their home.

One sunny morning, as Lily wandered deeper into the forest than ever before, she stumbled upon a mysterious old tree with a peculiar door carved into its trunk. She couldn't resist the urge to open the door and see what lay beyond. To her astonishment, the door led to a hidden realm filled with enchanting creatures, magical landscapes, and secrets waiting to be discovered.

Lily's arrival in the hidden realm was met with excitement and joy by the whimsical inhabitants. Fairies, talking animals, and mystical beings welcomed her with open arms. They explained that she was the "Chosen Explorer," destined to uncover the realm's deepest secrets and protect it from harm.

As Lily spent more time in this mystical world, she learned to harness its magic, gaining the ability to communicate with animals and understanding the ancient language of the trees. She embarked on thrilling quests, solved riddles, and helped her newfound friends in their time of need.

But the hidden realm was not without its challenges. A dark and sinister force, known as the "Shadow of Despair," threatened to engulf the magical world in darkness. With courage and determination, Lily and her friends joined forces to confront the shadow and protect their home.

In a climactic battle between light and darkness, Lily discovered that the true power of the realm lay within the hearts of those who believed in its magic. With the combined strength of her friends and the love they held for their world, they banished the Shadow of Despair and restored peace to the hidden realm.

Lily's time in the hidden realm had been an incredible adventure, but she knew that her place was back in the quiet village she called home. She bid her magical friends farewell, promising to return whenever they needed her. As she stepped back through the mysterious door in the tree, she felt grateful for the extraordinary experiences that had filled her life.

And so, Lily continued to explore the nearby forest, sharing her stories and adventures with the villagers, though they never knew the true extent of her magical journeys. In the end, she had found a world of enchantment right in her own backyard and had become the hero of a tale that would be passed down for generations in the quiet village."""
dynamic_detailed_summarize_premium(story4,context_type='story')

Keywords are-: ['lily', 'realm', 'hidden', 'door', 'friends']
Entities are-: ['the shadow of despair', 'one sunny morning', 'shadow of despair', 'lily']


"Once upon a time in a quiet village, there lived a young girl named Lily. Lily's arrival in the hidden realm was met with excitement and joy by the whimsical inhabitants. As Lily spent more time in this mystical world, she learned to harness its magic, gaining the ability to communicate with animals and understanding the ancient language of the trees. She embarked on thrilling quests, solved riddles, and helped her newfound friends in their time of need. Lily's time in the hidden realm had been an incredible adventure, but she knew that her place was back in the quiet village she called home."

In [49]:
dynamic_detailed_summarize_premium("""Climate change is one of the most pressing challenges of our time, with far-reaching consequences for ecosystems across the globe. Rising temperatures, extreme weather events, and shifting climate patterns are already affecting our planet's biodiversity.

One of the most visible impacts of climate change is the melting of polar ice caps and glaciers, leading to rising sea levels. This phenomenon poses a significant threat to coastal ecosystems and communities. Rising temperatures are also affecting marine life, causing coral bleaching and disrupting the food chains in the oceans.

In terrestrial ecosystems, many plant and animal species are facing the challenge of adapting to new climate conditions. Some may be forced to migrate to higher altitudes or latitudes in search of suitable habitats. This can lead to competition for resources and potential extinctions of less adaptable species.

Moreover, extreme weather events like hurricanes, droughts, and wildfires are becoming more frequent and intense due to climate change. These events not only directly impact ecosystems but also have far-reaching consequences for human populations.

Conservation efforts and sustainable practices are crucial in mitigating the effects of climate change on ecosystems. Protecting natural habitats, reducing greenhouse gas emissions, and supporting biodiversity conservation are vital steps in ensuring a healthier planet for future generations.

As the consequences of climate change continue to unfold, it's imperative that we act swiftly to address this global challenge and protect the diversity of life on Earth.""",1,'article')

Keywords are-: ['climate', 'change', 'ecosystems', 'rising', 'events']
Entities are-: ['earth', 'one']


'Climate change is one of the most pressing challenges of our time, with far-reaching consequences for ecosystems across the globe. These events not only directly impact ecosystems but also have far-reaching consequences for human populations.'

In [69]:
story5="""In the city of Marlowe, where the buildings touched the sky and the streets buzzed with the sound of flying cars, there lived an old watchmaker named Edmund. Edmund's shop was filled with timepieces of every kind, from pocket watches to grand wall clocks. But what made Edmund's shop truly unique was a little clockwork owl perched on a shelf behind the counter.

The townspeople were often curious about the owl. It was said that the owl was not just a mere timepiece but also possessed the wisdom of ages. Rumors spread that it could answer any question if you asked at the right time.

One day, a young girl named Lila entered the shop, her eyes wide with wonder. "Mr. Edmund," she began, "is it true that the owl knows everything?"

Edmund chuckled, "Well, not everything, Lila. But it does have some wisdom to share."

Lila, her curiosity piqued, asked, "How does it work?"

Edmund replied, "Every night, at exactly midnight, you may ask the owl one question, and it will give you an answer. But be cautious, for the owl speaks in riddles."

As the days passed, Lila became a regular at the shop, often stopping by to gaze at the clockwork owl and ponder the mysteries it held. One night, driven by a burning question, Lila sneaked into the shop at midnight. She asked the owl about the purpose of life.

The owl's gears whirred and clicked, and after what felt like an eternity, it spoke, "Life is a melody, ever-changing, never static. Seek your rhythm, find your tune, and dance to the beat of your own heart."

Lila left the shop that night, not with a clear answer, but with a newfound perspective on life. She realized that sometimes, the journey of seeking is more important than the answer itself.

Over time, the legend of the clockwork owl grew, drawing visitors from all over. But for Lila, the owl's wisdom was a guiding light, illuminating her path as she danced through life."""
dynamic_detailed_summarize_premium(story5)

Keywords are-: ['owl', 'lila', 'edmund', 'shop', 'answer']
Entities are-: ['edmund', 'lila', 'one', 'the days', 'marlowe', 'midnight', 'exactly midnight', 'one day', 'one night']


'In the city of Marlowe, where the buildings touched the sky and the streets buzzed with the sound of flying cars, there lived an old watchmaker named Edmund. One day, a young girl named Lila entered the shop, her eyes wide with wonder. Edmund chuckled, "Well, not everything, Lila.'

In [74]:
sample_text = """
In recent years, the effects of the digital divide have become increasingly apparent. As more and more essential services move online, people without internet access find it harder to participate in the modern economy. In response, many governments are considering subsidizing internet access. The idea is to ensure that everyone, regardless of their income or where they live, can access the digital world. 

Proponents argue that subsidized internet access can have multiple benefits. Firstly, it can lead to increased economic activity. As people gain access to online services, they can participate in e-commerce, take online courses, and apply for jobs online. Furthermore, it can help bridge the educational gap. Students in areas with poor or no internet access often lag behind their peers in terms of academic achievement. Having consistent internet access can help these students catch up.

However, critics of subsidized internet access point out potential drawbacks. The cost of implementing such programs can be high, and it's not clear whether the benefits would outweigh the costs. There's also concern about whether the subsidies would lead to a dependence on government support. Instead, some suggest that market solutions, like competition among internet service providers, might be a more effective way to bring down costs and increase access.

In conclusion, while the idea of subsidized internet access is well-intentioned, it's crucial to carefully weigh its potential benefits and drawbacks. The goal should be to ensure that everyone has a fair shot at participating in the digital age, but how best to achieve that is up for debate.
"""

# Using the function
# summary = dynamic_detailed_summarize_premium(sample_text, detail_level=1, context_type='article')
# summary
dynamic_detailed_summarize_premium("""Over the past two decades, the world has witnessed a massive shift in energy consumption and production patterns. The urgency to combat climate change and the economic advantages of renewable sources have both played pivotal roles in this transition.

Solar energy, in particular, has seen a meteoric rise. With advancements in solar panel efficiency and decreases in manufacturing costs, countries around the world are rapidly installing solar farms. The Sahara Desert, with its vast expanses of sunlight, has often been cited as an ideal location for solar energy capture.

Wind energy isn't far behind. Coastal areas and open plains have become hotspots for wind farms, taking advantage of persistent wind patterns. Innovations in turbine design have also allowed for more efficient energy capture.

Hydroelectric power, while not new, has gained renewed attention. Dams are not just about water storage anymore; they're powerhouses, quite literally. However, they've also been at the center of environmental and displacement debates.

While the advantages of renewable energy are many, there are challenges. Storage is a significant issue. The sun doesn't always shine, and the wind doesn't always blow. This intermittency means that there needs to be efficient ways to store energy for off-peak times. Battery technology, therefore, is a field seeing massive investment and research.

Another challenge is infrastructure. Many of our current grids are not designed to handle the influx of renewable energy, and there's a dire need to upgrade them. This requires not just funds, but also policy changes.

Lastly, while renewable energy is more sustainable, there are environmental concerns. Wind farms can affect local fauna, and the manufacture of solar panels isn't without its carbon footprint.

In conclusion, while the shift to renewable energy is promising and necessary, it's not a silver bullet. The challenges need addressing with the same fervor as the adoption of the energy sources themselves.

""",1,'article')

Keywords are-: ['energy', 'solar', 'wind', 'renewable', 'farms']
Entities are-: ['hydroelectric', 'the sahara desert', 'the past two decades']


"Over the past two decades, the world has witnessed a massive shift in energy consumption and production patterns. With advancements in solar panel efficiency and decreases in manufacturing costs, countries around the world are rapidly installing solar farms. Coastal areas and open plains have become hotspots for wind farms, taking advantage of persistent wind patterns. In conclusion, while the shift to renewable energy is promising and necessary, it's not a silver bullet."

In [77]:
# Sample text for testing
sample_text = """
Artificial Intelligence (AI) is a domain that has a long history but is still constantly and actively growing and changing. 
In this course, you'll learn the basics of modern AI as well as some of the representative applications of AI. 
Along the way, we also hope to excite you about the numerous applications and huge possibilities in the field of AI, 
which continues to expand human capability beyond our imagination. 
Today, with the advancements in the deep learning domain, we're witnessing a new revolution in AI, making it one of the most influential technologies of our time.
"""

# Testing the dynamic_detailed_summarize_premium function
summary = dynamic_detailed_summarize_premium(sample_text, detail_level=2, context_type='article')
summary


Keywords are-: ['ai', 'applications', 'domain', 'witnessing', 'course']
Entities are-: ['ai', 'today', 'artificial intelligence']


"\nArtificial Intelligence (AI) is a domain that has a long history but is still constantly and actively growing and changing. Today, with the advancements in the deep learning domain, we're witnessing a new revolution in AI, making it one of the most influential technologies of our time."

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize
import numpy as np
import spacy

# Function to extract important keywords using TF-IDF
def extract_important_keywords(input_text, num_keywords=5):
    my_stop_words = list(ENGLISH_STOP_WORDS)
    tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words, min_df=1, ngram_range=(1,2))
    tfidf_matrix = tfidf_vectorizer.fit_transform([input_text])
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    tfidf_values = tfidf_matrix.toarray()[0]
    sorted_indices = tfidf_values.argsort()[::-1]
    top_keywords = feature_names[sorted_indices][:num_keywords]
    return top_keywords.tolist()

# Function to extract named entities using SpaCy
def extract_named_entities(input_text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(input_text)
    named_entities = [ent.text.lower() for ent in doc.ents]
    return list(set(named_entities))

# Function to generate dynamic detailed summaries
def improved_dynamic_detailed_summarize(input_text, detail_level=1, context_type='general', important_keywords=None, important_entities=None):
    if important_keywords is None:
        important_keywords = extract_important_keywords(input_text)
    print(f'Important Keywords are-:{important_keywords}')
    if important_entities is None:
        important_entities = extract_named_entities(input_text)

    print(f'Important Entities are-:{important_keywords}')


    # Combine keywords and named entities
    important_words = set(important_keywords + important_entities)
    
    sentences = sent_tokenize(input_text)
    
    # Score sentences based on the number of important words and their position in the original text
    sentence_scores = [(i, sum(1 for word in important_words if word in sentence.lower())) for i, sentence in enumerate(sentences)]
    sorted_sentence_scores = sorted(sentence_scores, key=lambda x: (-x[1], x[0]))
    
    # Context weightage for adjusting the number of sentences
    context_weightage = {
        'general': 1,
        'story': 1.5,
        'report': 1.2,
        'article': 1.3
    }
    
    num_output_sentences = max(2, int(detail_level * len(sentences) / 5 * context_weightage.get(context_type, 1)))
    selected_sentences_indices = [score[0] for score in sorted_sentence_scores[:num_output_sentences]]
    
    # Sort the sentences based on their original order
    selected_sentences = [sentences[i] for i in sorted(selected_sentences_indices)]
    
    return ' '.join(selected_sentences)


In [86]:
improved_dynamic_detailed_summarize(article6,0.3,'article')

Important Keywords are-:['gaza', 'tech', 'people', 'said', 'companies']
Important Entities are-:['leen abubaker', 'manara', 'a few hours', 'the morning', 'around 100', 'years', 'silicon valley', 'silicon valley/europe', 'gazan', 'hacker news', 'linkedin', 'apple', 'jerusalem', 'instagram of them', '1,300', 'at least 150', '2', 'about 1.1 million', 'eric ries', 'nvidia', 'northern gaza', 'the middle frame', 'reid hoffman', 'first', 'the coming weeks', 'google and datadog', 'riyadh', 'ai', 'whatsapp', 'techcrunch', 'at least 2008', '1,500', 'the gaza sky geeks', 'forever', '200', 'daily', 'today', 'raya fatayer', 'jordan', 'january', 'tuesday', 'mai temraz', 'one', 'ibtikar fund', 'sellenvo', 'three', 'dalia awad', 'datadog', 'the west bank', 'one point #1', 'arab', 'marc benioff', 'mercy corps', 'a year ago', 'olivery', 'paul graham', 'sky geeks', 'gsg', 'seedcamp', '10 minutes', 'burj al-wattan', 'several days', 'belfast', '2022', 'ramallah', 'tonight', 'coretava', 'gaza city', 'gaza',

'Speaking to Ryan Sturgill, an American national and former head of the Gaza Sky Geeks accelerator run by sponsor Mercy Corps, and NGO aid organisation, the situation on the ground appears dire, after waves of shelling by the Israeli military. High growth companies emerging from Palestine include Menalytics (data analytics, invested in by Flat 6 labs); Olivery (last mile logistics, Flat6Labs and Ibtikar Fund); Coretava (employee and customer loyalty); and Sellenvo (an Amazon fulfillment partner). I feel like the situation is going to get significantly worse there over the coming weeks.”\nIliana Montauk, is co-Founder and CEO of Manara — a social impact startup funded by Y Combinator, Seedcamp, Reid Hoffman, Eric Ries, Marc Benioff, Paul Graham, and Jessica Livingston, among others — told Techcrunch via email that connectivity has decreased significantly in the past 24 hours. Dalia Awad, whose Medium post about getting into Google from Gaza went viral in 2021 (it was at one point #1 on 

In [178]:
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_sm")

def improved_dynamic_detailed_summarize(text, detail_level, context_type):
    # Preprocess the text
    def preprocess(text):
        # Tokenization and lowercasing
        tokens = [word.text.lower() for word in nlp(text)]
        return tokens

    word_count = len(preprocess(text))
    
    # Set the min_df_value and max_df_value based on word count
    min_df_value = 0.1 if word_count < 300 else 0.05
    max_df_value = 0.85 if word_count < 300 else 0.9
    
    # Extract keywords using TF-IDF
    vectorizer = TfidfVectorizer(max_df=max_df_value, min_df=min_df_value, stop_words='english')
    try:
        vectors = vectorizer.fit_transform([text])
    except ValueError:
        vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.01, stop_words='english')
        vectors = vectorizer.fit_transform([text])
        
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense().tolist()[0]
    keyword_scores = dict(zip(feature_names, dense))
    
    # Extract top N keywords based on word count
    N = 5 + (word_count // 300)
    top_keywords = [item[0] for item in sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:N]]
    
    # Named Entity Recognition using Spacy
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    entities=list(set(entities))
    # Sentence Tokenization
    sentences = [sent.text for sent in doc.sents]
    
    # Calculate Sentence Scores
    sentence_scores = []
    for sent in sentences:
        score = 0
        for word in preprocess(sent):
            if word in top_keywords:
                score += keyword_scores.get(word, 0)
            if word in entities:
                score += 2  # Giving more weightage to entities
        sentence_scores.append(score)
    
    # Dynamic Detail Level Adjustment
    num_sentences = {
        'short': int(len(sentences) * 0.2),
        'medium': int(len(sentences) * 0.5),
        'long': int(len(sentences) * 0.8)
    }
    detail_mapping = {
        1: 'short',
        2: 'medium',
        3: 'long'
    }
    context_weightage = {
        'story': 1.2,
        'article': 1,
        'report': 0.8
    }
    
    adjusted_num_sentences = int(num_sentences[detail_mapping[detail_level]] * context_weightage[context_type])
    adjusted_num_sentences = min(max(adjusted_num_sentences, 1), len(sentences))
    
    # Get the summary
    ranked_sentences = [sentences[i] for i in np.argsort(sentence_scores, axis=0)[-adjusted_num_sentences:]]
    summary = ' '.join(ranked_sentences)
    
    print(f"Important Keywords-{top_keywords}\nImportant_entities-{entities}")
    return summary
# You can use the function like this:
# improved_dynamic_detailed_summarize(your_text, detail_level, context_type)


In [179]:
# import spacy
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# nlp = spacy.load("en_core_web_sm")

# def preprocess(text):
#     doc = nlp(text)
#     return [token.text for token in doc if not token.is_stop and not token.is_punct]

# def improved_dynamic_detailed_summarize(text, detail_level, context_type):
#     word_count = len(text.split())
    
#     # 1. Keyword Extraction
#     min_df_value = 0.1 if word_count < 300 else 0.05
#     max_df_value = 0.85 if word_count < 300 else 0.9
#     step = 0.05
    
#     while True:
#         try:
#             vectorizer = TfidfVectorizer(max_df=max_df_value, min_df=min_df_value, stop_words='english')
#             vectors = vectorizer.fit_transform([text])
#             break
#         except ValueError:
#             min_df_value -= step
#             max_df_value += step

#     feature_names = vectorizer.get_feature_names_out()
#     dense = vectors.todense().tolist()[0]
#     keyword_list = [feature_names[i] for i in dense.argsort()[-5:][::-1]]
    
#     # If no keywords were extracted using TF-IDF, use term frequency
#     if not keyword_list:
#         term_freq = {}
#         for word in preprocess(text):
#             term_freq[word] = term_freq.get(word, 0) + 1
#         keyword_list = sorted(term_freq, key=term_freq.get, reverse=True)[:5]

#     # 2. Extract entities
#     doc = nlp(text)
#     entity_list = list(set([ent.text for ent in doc.ents]))

#     # 3. Sentence Tokenization and Ranking
#     sentences = [sent.string.strip() for sent in doc.sents]
#     vectors = vectorizer.transform(sentences)
#     cosine_matrix = cosine_similarity(vectors)
#     scores = cosine_matrix.sum(axis=1)
#     ranked_sentences = [sentences[i] for i in scores.argsort()[-int(detail_level * word_count/100):][::-1]]

#     summary = "\n\n".join(ranked_sentences)

#     return f"Important Keywords-{keyword_list}\nImportant_entities-{entity_list}\n{summary}"

# # Test
# story7 = """Deep in the heart of the Whispering Woods, where the trees sang tales of ancient times and the wind whispered secrets, lived a young woman named Elara. She wasn't like the villagers who lived on the outskirts of the forest; she could communicate with the creatures of the woods and understand the songs of the trees. One day, as Elara was strolling through the forest, she stumbled upon a wounded unicorn. Its horn was broken, and it lay in pain. Without hesitation, Elara approached the magnificent creature, whispering soothing words and using herbs to heal its wounds. In gratitude, the unicorn revealed its secret: it was the guardian of a magical portal that led to another realm. However, the portal had been sealed by dark forces, and the unicorn's horn was the key to unlocking it. But with its horn broken, the path to the other realm was lost. Determined to help, Elara set out on a quest to find a way to mend the unicorn's horn and reopen the portal. Her journey took her to the mountains where she met a wise old dragon, to the depths of the oceans where she befriended mermaids, and to the skies where she danced with the stars. Each encounter taught her a valuable lesson and brought her closer to her goal. After many adventures and challenges, Elara finally discovered a way to restore the unicorn's horn using the power of pure love. With the portal reopened, the realms were once again connected, and peace was restored. Elara returned to the Whispering Woods, not just as a friend of the forest, but as its savior. The trees sang her praises, and the creatures celebrated her bravery. And so, in the heart of the forest, the legend of Elara, the guardian of the realms, was born."""

# print(improved_dynamic_detailed_summarize(story7, 2, 'story'))


In [180]:
improved_dynamic_detailed_summarize(story7,1,'story')

Important Keywords-['elara', 'horn', 'unicorn', 'forest', 'portal', 'trees']
Important_entities-['Elara', 'the Whispering Woods', 'One day']


"One day, as Elara was strolling through the forest, she stumbled upon a wounded unicorn. After many adventures and challenges, Elara finally discovered a way to restore the unicorn's horn using the power of pure love. Determined to help, Elara set out on a quest to find a way to mend the unicorn's horn and reopen the portal."

In [94]:
improved_dynamic_detailed_summarize("""In the quiet town of Elderton, nestled amidst rolling hills and dense forests, there was a mystery that had persisted for generations. At the heart of the town stood the Elderton Mansion, a grand old house that was said to be haunted. Built in the 1800s by Sir Reginald Elderton, the mansion had been the site of numerous inexplicable occurrences, from ghostly apparitions to strange noises in the dead of night.

Sir Reginald had been a renowned explorer, traveling to the farthest corners of the world in search of ancient artifacts and hidden treasures. On one fateful expedition to the Amazon rainforest, it was said that he had found a mysterious artifact, a golden amulet with the power to bridge the world of the living and the dead. When he returned to Elderton, he was a changed man. Obsessed with the amulet's power, he locked himself in the mansion, conducting strange rituals and experiments.

One stormy night, a loud explosion echoed from the mansion, and Sir Reginald was never seen again. The amulet, too, vanished, fueling rumors that it had claimed him. The mansion was locked up, and as years turned into decades, it fell into disrepair. Yet, locals reported seeing lights in the mansion's windows and hearing Sir Reginald's voice on stormy nights.

Enter Clara Thompson, a young journalist with a penchant for the paranormal. Fascinated by the legend of Elderton Mansion, she decided to spend a night there, hoping to uncover its secrets. Armed with a camera, a flashlight, and her wits, she ventured into the mansion.

Inside, the air was thick with dust and age. Clara's flashlight beam cut through the darkness, revealing ornate furniture covered in dust cloths, and walls adorned with faded portraits of the Elderton lineage. As she explored, she stumbled upon Sir Reginald's study, a room filled with books, maps, and a journal detailing his obsession with the amulet.

As midnight approached, Clara began to hear whispers. Following the sound, she came to a grand ballroom, where a ghostly figure danced alone. It was Sir Reginald. He spoke to Clara, revealing that he had been trapped in a limbo between worlds by the amulet's power. He had been trying to harness its energy to communicate with his deceased wife, but instead, it had ensnared him. He implored Clara to find the amulet and free him.

Using clues from Sir Reginald's journal, Clara embarked on a quest that took her to hidden chambers and secret passages within the mansion. Finally, in the mansion's depths, she found the amulet, glowing with an otherworldly light. Remembering the rituals detailed in the journal, Clara performed a ceremony to break the amulet's curse.

As dawn broke, the mansion was silent. Clara emerged, the amulet in her possession, and Sir Reginald's spirit finally at peace. The mystery of Elderton had been solved, and Clara had a story that would make her career.""",1,,'story')

Important Keywords-:['mansion', 'amulet', 'clara', 'reginald', 'sir']
Important_entities-:['one', 'elderton', 'night']


'In the quiet town of Elderton, nestled amidst rolling hills and dense forests, there was a mystery that had persisted for generations. At the heart of the town stood the Elderton Mansion, a grand old house that was said to be haunted. The mystery of Elderton had been solved, and Clara had a story that would make her career.'

In [152]:
Important Keywords-['elara', 'horn', 'unicorn', 'forest', 'portal', 'trees']
Important_entities-['Elara', 'the Whispering Woods', 'One day']
"Without hesitation, Elara approached the magnificent creature, whispering soothing words and using herbs to heal its wounds. In gratitude, the unicorn revealed its secret: it was the guardian of a magical portal that led to another realm.\n\n Deep in the heart of the Whispering Woods, where the trees sang tales of ancient times and the wind whispered secrets, lived a young woman named Elara. Elara returned to the Whispering Woods, not just as a friend of the forest, but as its savior. And so, in the heart of the forest, the legend of Elara, the guardian of the realms, was born.\n\n However, the portal had been sealed by dark forces, and the unicorn's horn was the key to unlocking it. One day, as Elara was strolling through the forest, she stumbled upon a wounded unicorn. After many adventures and challenges, Elara finally discovered a way to restore the unicorn's horn using the power of pure love. Determined to help, Elara set out on a quest to find a way to mend the unicorn's horn and reopen the portal.\n\n"
    print(f"Important Keywords-{top_keywords}\nImportant_entities-{entities}")
    return summary

# You can use the function like this:
# improved_dynamic_detailed_summarize(your_text, detail_level, context_type)


In [164]:
story7="""Deep in the heart of the Whispering Woods, where the trees sang tales of ancient times and the wind whispered secrets, lived a young woman named Elara. She wasn't like the villagers who lived on the outskirts of the forest; she could communicate with the creatures of the woods and understand the songs of the trees.

One day, as Elara was strolling through the forest, she stumbled upon a wounded unicorn. Its horn was broken, and it lay in pain. Without hesitation, Elara approached the magnificent creature, whispering soothing words and using herbs to heal its wounds. In gratitude, the unicorn revealed its secret: it was the guardian of a magical portal that led to another realm.

However, the portal had been sealed by dark forces, and the unicorn's horn was the key to unlocking it. But with its horn broken, the path to the other realm was lost. Determined to help, Elara set out on a quest to find a way to mend the unicorn's horn and reopen the portal.

Her journey took her to the mountains where she met a wise old dragon, to the depths of the oceans where she befriended mermaids, and to the skies where she danced with the stars. Each encounter taught her a valuable lesson and brought her closer to her goal.

After many adventures and challenges, Elara finally discovered a way to restore the unicorn's horn using the power of pure love. With the portal reopened, the realms were once again connected, and peace was restored.

Elara returned to the Whispering Woods, not just as a friend of the forest, but as its savior. The trees sang her praises, and the creatures celebrated her bravery. And so, in the heart of the forest, the legend of Elara, the guardian of the realms, was born.

"""
improved_dynamic_detailed_summarize(story7,2,'story')

Important Keywords-['elara', 'horn', 'unicorn', 'forest', 'portal', 'trees']
Important_entities-['Elara', 'the Whispering Woods', 'One day']


"Without hesitation, Elara approached the magnificent creature, whispering soothing words and using herbs to heal its wounds. In gratitude, the unicorn revealed its secret: it was the guardian of a magical portal that led to another realm.\n\n Deep in the heart of the Whispering Woods, where the trees sang tales of ancient times and the wind whispered secrets, lived a young woman named Elara. Elara returned to the Whispering Woods, not just as a friend of the forest, but as its savior. And so, in the heart of the forest, the legend of Elara, the guardian of the realms, was born.\n\n However, the portal had been sealed by dark forces, and the unicorn's horn was the key to unlocking it. One day, as Elara was strolling through the forest, she stumbled upon a wounded unicorn. After many adventures and challenges, Elara finally discovered a way to restore the unicorn's horn using the power of pure love. Determined to help, Elara set out on a quest to find a way to mend the unicorn's horn an

In [145]:
dynamic_detailed_summarize_premium(story7,1,'story')

Keywords are-: ['elara', 'unicorn', 'horn', 'portal', 'forest']
Entities are-: ['elara', 'one day', 'the whispering woods']


"Deep in the heart of the Whispering Woods, where the trees sang tales of ancient times and the wind whispered secrets, lived a young woman named Elara. She wasn't like the villagers who lived on the outskirts of the forest; she could communicate with the creatures of the woods and understand the songs of the trees. Elara returned to the Whispering Woods, not just as a friend of the forest, but as its savior."

In [183]:
improved_dynamic_detailed_summarize("""Artificial intelligence (AI) is revolutionizing industries, enhancing productivity, and influencing the way we work and interact. As AI continues to advance, its potential impact on society and the role it will play in the future remains a hot topic of discussion.

Historically, AI was conceptualized as machines that could mimic cognitive functions such as learning and problem-solving. Today, AI is more than just robotics. With the rise of machine learning algorithms and vast amounts of data, AI systems can perform tasks that previously required human intelligence. These include speech recognition, decision-making, and language translation.

Big tech companies like Google, Apple, and Microsoft are heavily investing in AI research and development. Google's DeepMind, for instance, developed an AI named AlphaGo that defeated the world champion in the game of Go, a feat many thought was decades away. Similarly, Microsoft's AI has achieved human parity in translating news articles from Chinese to English.

However, with great power comes great responsibility. As AI systems become more integrated into our daily lives, concerns about AI ethics, privacy, and potential misuse have arisen. Critics argue that AI could be used for malicious purposes, such as creating deepfakes or autonomous weapons. Additionally, there are fears about job losses in sectors where AI can automate tasks.

On the positive side, AI has the potential to solve some of the world's most pressing challenges. In healthcare, AI algorithms can predict outbreaks, personalize medical treatments, and assist in drug discovery. In environmental science, AI can monitor deforestation, track wildlife, and predict natural disasters.

To harness the full potential of AI, collaboration between governments, industries, and academia is crucial. Proper regulations and ethical guidelines can ensure that AI benefits humanity and does not harm it. As we stand on the cusp of the AI revolution, it's essential to move forward with caution, optimism, and a focus on building a future that benefits all.""",2,'article')

Important Keywords-['ai', 'potential', 'algorithms', 'benefits', 'future', 'google']
Important_entities-['Microsoft', 'English', 'Google', 'Today', 'Chinese', "Google's", 'the AI revolution', 'decades', 'AlphaGo', 'DeepMind', 'Apple', 'AI']


"Proper regulations and ethical guidelines can ensure that AI benefits humanity and does not harm it. In healthcare, AI algorithms can predict outbreaks, personalize medical treatments, and assist in drug discovery. Big tech companies like Google, Apple, and Microsoft are heavily investing in AI research and development. On the positive side, AI has the potential to solve some of the world's most pressing challenges. To harness the full potential of AI, collaboration between governments, industries, and academia is crucial. As we stand on the cusp of the AI revolution, it's essential to move forward with caution, optimism, and a focus on building a future that benefits all. As AI continues to advance, its potential impact on society and the role it will play in the future remains a hot topic of discussion.\n\n As AI systems become more integrated into our daily lives, concerns about AI ethics, privacy, and potential misuse have arisen. Google's DeepMind, for instance, developed an AI n

In [186]:
dynamic_detailed_summarize_premium("""Artificial intelligence (AI) is revolutionizing industries, enhancing productivity, and influencing the way we work and interact. As AI continues to advance, its potential impact on society and the role it will play in the future remains a hot topic of discussion.

Historically, AI was conceptualized as machines that could mimic cognitive functions such as learning and problem-solving. Today, AI is more than just robotics. With the rise of machine learning algorithms and vast amounts of data, AI systems can perform tasks that previously required human intelligence. These include speech recognition, decision-making, and language translation.

Big tech companies like Google, Apple, and Microsoft are heavily investing in AI research and development. Google's DeepMind, for instance, developed an AI named AlphaGo that defeated the world champion in the game of Go, a feat many thought was decades away. Similarly, Microsoft's AI has achieved human parity in translating news articles from Chinese to English.

However, with great power comes great responsibility. As AI systems become more integrated into our daily lives, concerns about AI ethics, privacy, and potential misuse have arisen. Critics argue that AI could be used for malicious purposes, such as creating deepfakes or autonomous weapons. Additionally, there are fears about job losses in sectors where AI can automate tasks.

On the positive side, AI has the potential to solve some of the world's most pressing challenges. In healthcare, AI algorithms can predict outbreaks, personalize medical treatments, and assist in drug discovery. In environmental science, AI can monitor deforestation, track wildlife, and predict natural disasters.

To harness the full potential of AI, collaboration between governments, industries, and academia is crucial. Proper regulations and ethical guidelines can ensure that AI benefits humanity and does not harm it. As we stand on the cusp of the AI revolution, it's essential to move forward with caution, optimism, and a focus on building a future that benefits all.""",0.5,'article')

Keywords are-: ['ai', 'potential', 'world', 'tasks', 'algorithms']
Entities are-: ['alphago', 'deepmind', 'google', 'chinese', 'ai', 'today', 'microsoft', 'decades', 'the ai revolution', 'apple', 'english', "google's"]


'Artificial intelligence (AI) is revolutionizing industries, enhancing productivity, and influencing the way we work and interact. To harness the full potential of AI, collaboration between governments, industries, and academia is crucial.'

In [190]:
gen_ai_article="""The topic of generative AI comes up frequently in my newsletter, Actuator. I admit that I was a bit hesitant to spend more time on the subject a few months back. Anyone who has been reporting on technology for as long as I have has lived through countless hype cycles and been burned before. Reporting on tech requires a healthy dose of skepticism, hopefully tempered by some excitement about what can be done.

This time out, it seemed generative AI was waiting in the wings, biding its time, waiting for the inevitable cratering of crypto. As the blood drained out of that category, projects like ChatGPT and DALL-E were standing by, ready to be the focus of breathless reporting, hopefulness, criticism, doomerism and all the different Kübler-Rossian stages of the tech hype bubble.

Those who follow my stuff know that I was never especially bullish on crypto. Things are, however, different with generative AI. For starters, there’s a near universal agreement that artificial intelligence/machine learning broadly will play more centralized roles in our lives going forward.

Smartphones offer great insight here. Computational photography is something I write about somewhat regularly. There have been great advances on that front in recent years, and I think many manufacturers have finally struck a good balance between hardware and software when it comes to both improving the end product and lowering the bar of entry. Google, for instance, pulls off some truly impressive tricks with editing features like Best Take and Magic Eraser.

Sure, they’re neat tricks, but they’re also useful, rather than being features for features’ sake. Moving forward, however, the real trick will be seamlessly integrating them into the experience. With ideal future workflows, most users will have little to no notion of what’s happening behind the scenes. They’ll just be happy that it works. It’s the classic Apple playbook.

Generative AI offers a similar “wow” effect out the gate, which is another way it differs from its hype cycle predecessor. When your least tech savvy relative can sit at a computer, type a few words into a dialogue field and then watch as the black box spits out paintings and short stories, there isn’t much conceptualizing required. That’s a big part of the reason all of this caught on as quickly as it did — most times when everyday people get pitched cutting-edge technologies, it requires them to visualize how it might look five or 10 years down the road.

With ChatGPT, DALL-E, etc., you can experience it firsthand right now. Of course, the flip side of this is how difficult it becomes to temper expectations. Much as people are inclined to imbue robots with human or animal intelligence, without a fundamental understanding of AI, it’s easy to project intentionality here. But that’s just how things go now. We lead with the attention-grabbing headline and hope people stick around long enough to read about machinations behind it.

Spoiler alert: Nine times out of 10 they won’t, and suddenly we’re spending months and years attempting to walk things back to reality.

One of the nice perks of my job is the ability to break these things down with people much smarter than me. They take the time to explain things and hopefully I do a good job translating that for readers (some attempts are more successful than others).

Once it became clear that generative AI has an important role to play in the future of robotics, I’ve been finding ways to shoehorn questions into conversations. I find that most people in the field agree with the statement in the previous sentence, and it’s fascinating to see the breadth of impact they believe it will have.

For example, in my recent conversation with Marc Raibert and Gill Pratt, the latter explained the role generative AI is playing in its approach to robot learning:

We have figure out how to do something, which is use modern generative AI techniques that enable human demonstration of both position and force to essentially teach a robot from just a handful of examples. The code is not changed at all. What this is based on is something called diffusion policy. It’s work that we did in collaboration with Columbia and MIT. We’ve taught 60 different skills so far.

Last week, when I asked Nvidia’s VP and GM of Embedded and Edge Computing, Deepu Talla why the company believes generative AI is more than a fad, he told me:

I think it speaks in the results. You can already see the productivity improvement. It can compose an email for me. It’s not exactly right, but I don’t have to start from zero. It’s giving me 70%. There are obvious things you can already see that are definitely a step function better than how things were before. Summarizing something’s not perfect. I’m not going to let it read and summarize for me. So, you can already see some signs of productivity improvements.

Meanwhile, during my last conversation with Daniela Rus, the MIT CSAIL head explained how researchers are using generative AI to actually design the robots:

It turns out that generative AI can be quite powerful for solving even motion planning problems. You can get much faster solutions and much more fluid and human-like solutions for control than with model predictive solutions. I think that’s very powerful, because the robots of the future will be much less roboticized. They will be much more fluid and human-like in their motions.

We’ve also used generative AI for design. This is very powerful. It’s also very interesting , because it’s not just pattern generation for robots. You have to do something else. It can’t just be generating a pattern based on data. The machines have to make sense in the context of physics and the physical world. For that reason, we connect them to a physics-based simulation engine to make sure the designs meet their required constraints.

This week, a team at Northwestern University unveiled its own research into AI-generated robot design. The researchers showcased how they designed a “successfully walking robot in mere seconds.” It’s not much to look at, as these things go, but it’s easy enough to see how with additional research, the approach could be used to create more complex systems.

“We discovered a very fast AI-driven design algorithm that bypasses the traffic jams of evolution, without falling back on the bias of human designers,” said research lead Sam Kriegman. “We told the AI that we wanted a robot that could walk across land. Then we simply pressed a button and presto! It generated a blueprint for a robot in the blink of an eye that looks nothing like any animal that has ever walked the earth. I call this process ‘instant evolution.’”

It was the AI program’s choice to put legs on the small, squishy robot. “It’s interesting because we didn’t tell the AI that a robot should have legs,” Kriegman added. “It rediscovered that legs are a good way to move around on land. Legged locomotion is, in fact, the most efficient form of terrestrial movement.”

“From my perspective, generative AI and physical automation/robotics are what’s going to change everything we know about life on Earth,” Formant founder and CEO Jeff Linnell told me this week. “I think we’re all hip to the fact that AI is a thing and are expecting every one our jobs, every company and student will be impacted. I think it’s symbiotic with robotics. You’re not going to have to program a robot. You’re going to speak to the robot in English, request an action and then it will be figured out. It’s going to be a minute for that.”

"""
dynamic_detailed_summarize_premium(gen_ai_article,1,'article')

Keywords are-: ['ai', 'generative', 'robot', 'things', 'going']
Entities are-: ['60', 'zero', 'daniela rus', 'jeff linnell', 'columbia', 'ai', '70%', 'earth', 'recent years', 'years', 'magic eraser', 'a few months back', '10', 'months', 'best take', 'this week', 'apple', 'gill pratt', 'kriegman', 'one', 'northwestern university', 'five or 10 years', 'kübler-rossian', 'english', 'google', 'sam kriegman', 'actuator', 'nine', 'marc raibert', 'nvidia', 'deepu talla', 'seconds', 'gm', 'last week', 'mit']


'The topic of generative AI comes up frequently in my newsletter, Actuator. This time out, it seemed generative AI was waiting in the wings, biding its time, waiting for the inevitable cratering of crypto. Things are, however, different with generative AI. There have been great advances on that front in recent years, and I think many manufacturers have finally struck a good balance between hardware and software when it comes to both improving the end product and lowering the bar of entry. Generative AI offers a similar “wow” effect out the gate, which is another way it differs from its hype cycle predecessor. Once it became clear that generative AI has an important role to play in the future of robotics, I’ve been finding ways to shoehorn questions into conversations. For example, in my recent conversation with Marc Raibert and Gill Pratt, the latter explained the role generative AI is playing in its approach to robot learning:\n\nWe have figure out how to do something, which is use mo

In [192]:
improved_dynamic_detailed_summarize(gen_ai_article,1,'article')

Important Keywords-['ai', 'generative', 'robot', 'things', 'going', 'human', 'just', 'like', 'people', 'think']
Important_entities-['60', 'zero', 'Daniela Rus', 'Gill Pratt', 'Sam Kriegman', '70%', 'Earth', 'Magic Eraser', 'Kübler-Rossian', 'earth', 'Jeff Linnell', 'Northwestern University', 'recent years', 'years', 'a few months back', '10', 'months', 'One', 'this week', 'AI', 'Nine', 'Actuator', 'Columbia', 'Deepu Talla', 'This week', 'Google', 'five or 10 years', 'GM', 'MIT', 'Nvidia', 'Kriegman', 'Marc Raibert', 'English', 'Last week', 'seconds', 'Apple', 'Best Take']


'Generative AI offers a similar “wow” effect out the gate, which is another way it differs from its hype cycle predecessor. Last week, when I asked Nvidia’s VP and GM of Embedded and Edge Computing, Deepu Talla why the company believes generative AI is more than a fad, he told me:\n\nI think it speaks in the results. Things are, however, different with generative AI. Meanwhile, during my last conversation with Daniela Rus, the MIT CSAIL head explained how researchers are using generative AI to actually design the robots:\n\nIt turns out that generative AI can be quite powerful for solving even motion planning problems. It’s not exactly right, but I don’t have to start from zero. We’ve taught 60 different skills so far.\n\n I admit that I was a bit hesitant to spend more time on the subject a few months back. There have been great advances on that front in recent years, and I think many manufacturers have finally struck a good balance between hardware and software when it comes to both 

In [193]:
story8="""In the heart of a bustling town named Eldoria, there was a skilled clockmaker named Alistair. His shop, adorned with intricate clocks of all sizes, was a marvel for both locals and visitors. From grand grandfather clocks to delicate pocket watches, Alistair crafted them all.

    One day, a young girl named Lila entered the shop with a request. She held a broken pocket watch, passed down through generations in her family. The watch had stopped ticking, and she hoped Alistair could breathe life into it once more.

    Alistair, intrigued by the watch's unique design, agreed. As he opened the back, he discovered an encrypted note hidden inside. The note read, "To unlock time's secret, listen to its heartbeat."

    Puzzled, Alistair placed the watch near his ear. The rhythmic ticking began to form a melody. Entranced, he followed the tune's guidance and adjusted the watch's internal gears. To his amazement, the watch didn't just start ticking—it began projecting shimmering images of Lila's ancestors, revealing snippets of their lives.

    Lila watched in awe as the scenes unfolded: her great-grandfather as a brave soldier, her grandmother dancing under the moonlight, and her mother's first steps as a child. The watch was more than a timekeeper; it was a keeper of memories.

    Alistair returned the mended watch to Lila, who was teary-eyed with gratitude. She left the shop, holding not just a repaired watch, but a piece of her family's history.

    Rumors of the magical pocket watch spread throughout Eldoria. People from far and wide visited Alistair's shop, hoping to discover the secrets within their timepieces. But no other watch held the magic Lila's did.

    Years later, Alistair received a gift—a grand clock for his shop, sent anonymously. Every hour, on the hour, it played the same melody he had heard from Lila's watch. A note accompanied the clock: "To remind you that time is not just about ticking seconds, but about cherished moments. Thank you."

    Alistair smiled, realizing that the true magic wasn't in the watches he fixed, but in the memories and moments they represented.

"""


In [194]:
improved_dynamic_detailed_summarize(story8,1,'story')

Important Keywords-['watch', 'alistair', 'lila', 'shop', 'ticking', 'just']
Important_entities-['first', 'Puzzled, Alistair', 'the hour', 'One day', 'Lila', 'Every hour', 'Eldoria', 'Years later', 'Alistair']


'"\n\nPuzzled, Alistair placed the watch near his ear. Alistair, intrigued by the watch\'s unique design, agreed. To his amazement, the watch didn\'t just start ticking—it began projecting shimmering images of Lila\'s ancestors, revealing snippets of their lives.\n\n The watch had stopped ticking, and she hoped Alistair could breathe life into it once more.\n\n Alistair returned the mended watch to Lila, who was teary-eyed with gratitude. Lila watched in awe as the scenes unfolded: her great-grandfather as a brave soldier, her grandmother dancing under the moonlight, and her mother\'s first steps as a child.'

In [199]:
dynamic_detailed_summarize_premium(story8,0.8,'story')

Keywords are-: ['watch', 'alistair', 'lila', 'shop', 'ticking']
Entities are-: ['first', 'lila', 'alistair', 'every hour', 'years later', 'the hour', 'puzzled, alistair', 'eldoria', 'one day']


"In the heart of a bustling town named Eldoria, there was a skilled clockmaker named Alistair. One day, a young girl named Lila entered the shop with a request. Alistair, intrigued by the watch's unique design, agreed. Puzzled, Alistair placed the watch near his ear. Rumors of the magical pocket watch spread throughout Eldoria."

In [200]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize
import numpy as np

nlp = spacy.load("en_core_web_sm")

# Enhanced Sentence Scoring
def score_sentences(sentences, important_keywords, important_entities):
    keyword_weight = 1
    entity_weight = 1.5
    scores = []

    for sentence in sentences:
        score = 0
        tokens = sentence.split()
        for token in tokens:
            if token.lower() in important_keywords:
                score += keyword_weight
            if token.lower() in important_entities:
                score += entity_weight
        scores.append(score)

    return scores

# Simple Rule-Based Sentence Compression
def compress_sentence(sentence):
    doc = nlp(sentence)
    tokens = [token.text for token in doc if token.pos_ not in ['ADJ', 'ADV'] and token.dep_ != 'punct']
    compressed = ' '.join(tokens)
    return compressed

# Function to generate dynamic detailed summaries with the enhancements
def dynamic_detailed_summarize_premium1(input_text, detail_level=1, context_type='general'):
    important_keywords = extract_important_keywords(input_text)
    important_entities = extract_named_entities(input_text)
    
    print(f'Keywords are-: {important_keywords}')
    print(f'Entities are-: {important_entities}')    
    
    sentences = sent_tokenize(input_text)
    sentence_scores = score_sentences(sentences, important_keywords, important_entities)

    # Context weightage for adjusting the number of sentences based on context_type
    context_weightage = {
        'general': 1,
        'story': 1.5,
        'report': 1.2,
        'article': 1.3  # Adding support for 'article' context
    }

    num_output_sentences = max(2, int(detail_level * len(sentences) / 5 * context_weightage.get(context_type, 1)))
    
    # Selecting top-scoring sentences
    ranked_sentences = [sentences[i] for i in np.argsort(sentence_scores, axis=0)[-num_output_sentences:]]

    # Compressing the sentences
    compressed_sentences = [compress_sentence(sentence) for sentence in ranked_sentences]
    
    return ' '.join(compressed_sentences)


In [203]:
dynamic_detailed_summarize_premium1(story8,1.5,'story')

Keywords are-: ['watch', 'alistair', 'lila', 'shop', 'ticking']
Entities are-: ['first', 'lila', 'alistair', 'every hour', 'years later', 'the hour', 'puzzled, alistair', 'eldoria', 'one day']


"To his amazement the watch did n't start ticking it began projecting shimmering images of Lila 's ancestors revealing snippets of their lives The ticking began to form a melody The watch was than a timekeeper it was a keeper of memories Years Alistair received a gift a clock for his shop sent From grandfather clocks to pocket watches Alistair crafted them all Alistair smiled realizing that the magic was n't in the watches he fixed but in the memories and moments they represented Alistair returned the mended watch to Lila who was eyed with gratitude Puzzled Alistair placed the watch near his ear The watch had stopped ticking and she hoped Alistair could breathe life into it One day a girl named Lila entered the shop with a request Lila watched in awe as the scenes unfolded her grandfather as a soldier her grandmother dancing under the moonlight and her mother 's steps as a child"

In [205]:
dynamic_detailed_summarize_premium("""Once upon a time in a quaint, picturesque village nestled between rolling hills and a sparkling river, there lived a young girl named Lily. She was known throughout the village for her radiant smile and her insatiable curiosity. Lily's eyes, as bright as the morning sun, seemed to sparkle with wonder at the world around her.

Lily's favorite place to explore was the dense forest that bordered the village. The villagers often warned her about the dangers that lurked within the dark woods, but Lily was undeterred. She believed that there was more to the forest than met the eye, and she longed to discover its secrets.

One crisp autumn morning, as the leaves turned brilliant shades of red and gold, Lily decided to embark on a grand adventure into the heart of the forest. She packed a small satchel with some food, a lantern, and a notebook to document her findings. With a determined spirit and her loyal dog, Max, by her side, she ventured into the unknown.

As Lily and Max ventured deeper into the forest, they encountered a myriad of wonders. They found a hidden waterfall cascading gracefully into a crystal-clear pool, where colorful fish darted among the rocks. Further on, they stumbled upon a grove of ancient, towering trees, their branches forming a natural cathedral beneath the sky.

As the day turned to evening, Lily and Max came upon a clearing bathed in the soft glow of fireflies. In the center of the clearing, there stood an old, gnarled oak tree, its trunk adorned with a curious, ancient carving. Lily carefully examined the carving and sketched it in her notebook. It depicted a map, with a series of symbols and lines leading to a place unknown.

With excitement bubbling in her chest, Lily decided to follow the map. She traced the lines through the forest, each step taking her deeper into the wilderness. Along the way, she encountered talking animals, wise owls, and friendly sprites who helped her decipher the symbols on the map.

Finally, after days of wandering, Lily arrived at a hidden glen, bathed in the soft light of a magical, glowing flower. At the center of the glen stood a majestic, ancient tree unlike any other. Its bark shimmered with an otherworldly radiance, and its branches seemed to reach up to the heavens themselves.

Lily approached the tree with reverence, and as she touched its trunk, she felt a surge of energy flow through her. The tree whispered ancient wisdom, filling her with a deep understanding of the interconnectedness of all things in the world. She realized that the forest held not just secrets but also the key to a profound, unending adventure: the adventure of learning and understanding the world around her.

With a heart full of gratitude and knowledge, Lily bid farewell to the magical glen and made her way back home, her trusty companion Max by her side. She returned to her village, where she shared the tales of her incredible journey and the wisdom she had gained from the enchanted tree. The villagers were amazed by her stories and the lessons she imparted.

From that day on, Lily became known as the village's wisest explorer, and she continued to venture into the forest, discovering new wonders and sharing her knowledge with her fellow villagers. And so, in the heart of the forest, and in the heart of a young girl, the spirit of curiosity and the love of adventure lived on, forever inspiring those who followed in her footsteps.""",0.5,'story')

Keywords are-: ['lily', 'forest', 'tree', 'village', 'heart']
Entities are-: ['that day', 'one', 'evening', 'lily', 'max', 'the day', 'autumn morning', 'morning']


"Once upon a time in a quaint, picturesque village nestled between rolling hills and a sparkling river, there lived a young girl named Lily. Lily's favorite place to explore was the dense forest that bordered the village. And so, in the heart of the forest, and in the heart of a young girl, the spirit of curiosity and the love of adventure lived on, forever inspiring those who followed in her footsteps."

In [206]:
dynamic_detailed_summarize_premium("""Once upon a time, there were three little pigs who lived in a small village. Each pig had its own house to live in. The first pig was lazy and built his house out of straw because it was quick and easy. The second pig was a bit more diligent and constructed his house out of sticks. The third pig, however, was wise and hardworking, and he built his house using sturdy bricks.

One day, a big, bad wolf came to the village. He huffed and puffed and blew down the first little pig's straw house with ease. The frightened pig ran to the second pig's house, but the wolf, undeterred, huffed and puffed again and knocked down the stick house.

Now, the two terrified pigs ran to the third pig's brick house. The wolf tried with all his might to blow down the brick house, but no matter how hard he huffed and puffed, he couldn't knock it down. Frustrated and hungry, the wolf decided to climb down the chimney and into the house. However, the clever third pig had a pot of boiling water waiting in the fireplace. The wolf fell right into it and ran away, never to bother the three little pigs again.

The three little pigs celebrated their victory and learned an important lesson about the value of hard work and building things properly. From that day on, they lived happily in their sturdy brick house, knowing they were safe from the big, bad wolf.

And so, the tale of the Three Little Pigs teaches us the importance of diligence and hard work in the face of adversity.""",1,'story')

Keywords are-: ['house', 'pig', 'wolf', 'little', 'pigs']
Entities are-: ['that day', 'first', 'two', 'second', 'three', 'wolf', 'the three little pigs', 'one day', 'third']


'Once upon a time, there were three little pigs who lived in a small village. One day, a big, bad wolf came to the village. The wolf fell right into it and ran away, never to bother the three little pigs again. And so, the tale of the Three Little Pigs teaches us the importance of diligence and hard work in the face of adversity.'

In [208]:
dynamic_detailed_summarize_premium("""On a tranquil morning, the world awakens with a symphony of natural sounds. Birds greet the day with melodious songs, and the sun's first rays cast a warm, golden hue upon the landscape. Dew-kissed flowers bloom, and leaves rustle in the gentle breeze.

Nature's beauty is not confined to mornings alone. Throughout the day, rivers meander through lush valleys, their waters glistening under the sun's watchful eye. Majestic mountains stand as sentinels, their snow-capped peaks touching the sky. Forests are a sanctuary of green, where ancient trees whisper secrets of the past.

In this world of wonders, life flourishes in myriad forms. Creatures of all sizes roam the land, from the smallest insects to the mightiest predators. Each species plays its role in the intricate web of life, a dance of survival and cooperation.

Humanity, too, is part of this grand tapestry. Cities rise with towering skyscrapers, bustling with the energy of progress and innovation. People of diverse backgrounds and dreams navigate their way through the urban labyrinth, seeking connection and purpose.

Yet, amidst the modern world, there are moments of reflection and solace. Individuals find solace in the pages of books, the strokes of a paintbrush, or the gentle strumming of a guitar. Communities gather to celebrate traditions, share stories, and express the bonds of kinship.

As the day draws to a close, the world slips into the tranquil embrace of night. Stars twinkle in the vast expanse above, and the moon casts its gentle glow. It is a time for dreams, for contemplation, and for the world to rest in peaceful slumber.

In this ever-evolving world, the beauty of nature, the diversity of life, and the human spirit's resilience continue to weave a tapestry of experiences. It's a world where each day brings new discoveries, where challenges are met with courage, and where the wonders of existence are celebrated.
""",1)

Keywords are-: ['world', 'day', 'life', 'gentle', 'wonders']
Entities are-: ['first', 'night', 'the day', 'mornings', 'each day', 'nature', 'a tranquil morning']


'On a tranquil morning, the world awakens with a symphony of natural sounds. As the day draws to a close, the world slips into the tranquil embrace of night.'

In [209]:
dynamic_detailed_summarize_premium(story8,1.5,'story')

Keywords are-: ['watch', 'alistair', 'lila', 'shop', 'ticking']
Entities are-: ['first', 'lila', 'alistair', 'every hour', 'years later', 'the hour', 'puzzled, alistair', 'eldoria', 'one day']


"In the heart of a bustling town named Eldoria, there was a skilled clockmaker named Alistair. From grand grandfather clocks to delicate pocket watches, Alistair crafted them all. One day, a young girl named Lila entered the shop with a request. The watch had stopped ticking, and she hoped Alistair could breathe life into it once more. Alistair, intrigued by the watch's unique design, agreed. Puzzled, Alistair placed the watch near his ear. Alistair returned the mended watch to Lila, who was teary-eyed with gratitude. Rumors of the magical pocket watch spread throughout Eldoria. Years later, Alistair received a gift—a grand clock for his shop, sent anonymously."

In [210]:
dynamic_detailed_summarize_premium("""Impact of Climate Change on Global Agriculture

Executive Summary:
The ongoing effects of climate change present significant challenges to global agriculture. This report delves into the repercussions of rising temperatures, altered precipitation patterns, and increasing frequency of extreme weather events on agricultural productivity and food security.

1. Introduction:
Climate change, characterized by global warming, sea-level rise, and extreme weather events, has become one of the most pressing issues of the 21st century. Agriculture, being highly dependent on climatic conditions, stands vulnerable to these changes.

2. Key Findings:

Temperature Rise: A consistent increase in temperatures affects crop germination and growth. While certain crops may benefit from a slight temperature rise, most will suffer, especially if the increase surpasses their optimal growth temperature.

Precipitation Patterns: Changes in rainfall patterns can lead to prolonged droughts or excessive rainfall. Both scenarios are detrimental to crops, impacting yield and quality.

Extreme Weather Events: The rising frequency of hurricanes, floods, and droughts can devastate crops, leading to food shortages and market instability.

Shift in Agricultural Zones: As temperatures rise, traditional agricultural zones are shifting. This forces farmers to adapt by either changing their cropping patterns or relocating their farming operations.

3. Economic Implications:
The agricultural sector, contributing significantly to the GDP of many nations, faces economic strain due to reduced yields. This leads to increased food prices, affecting consumers worldwide.

4. Social Impact:
Food security is compromised, especially in developing nations. This can lead to malnutrition, societal unrest, and even mass migrations in search of better livelihood opportunities.

5. Potential Solutions:

Adaptive Farming Techniques: Introduction of drought-resistant crops, efficient irrigation systems, and soil conservation methods.

Early Warning Systems: Implementation of meteorological tools to forecast extreme weather events, allowing farmers to take preventive measures.

Governmental Support: Policy interventions, subsidies, and support in research and development to drive innovations in farming.

6. Conclusion:
While the challenges posed by climate change are daunting, with collective effort, technology, and adaptive strategies, the global agricultural community can navigate these uncharted waters. Continuous research, investment, and awareness are crucial.""",1,'report')

Keywords are-: ['agricultural', 'crops', 'events', 'extreme', 'food']
Entities are-: ['1', '5', 'the 21st century', '4', 'precipitation patterns: changes', 'governmental support', 'extreme weather', '2', 'potential solutions', 'adaptive farming techniques: introduction of drought', '6', 'continuous', 'early warning systems: implementation of meteorological', '3']


'This report delves into the repercussions of rising temperatures, altered precipitation patterns, and increasing frequency of extreme weather events on agricultural productivity and food security. Introduction:\nClimate change, characterized by global warming, sea-level rise, and extreme weather events, has become one of the most pressing issues of the 21st century. Shift in Agricultural Zones: As temperatures rise, traditional agricultural zones are shifting. Social Impact:\nFood security is compromised, especially in developing nations. Conclusion:\nWhile the challenges posed by climate change are daunting, with collective effort, technology, and adaptive strategies, the global agricultural community can navigate these uncharted waters.'

In [211]:
dynamic_detailed_summarize_premium("""In the last decade, renewable energy has emerged from the fringes to become a mainstream powerhouse. From solar panels adorning rooftops to wind turbines dotting landscapes, the evidence of a seismic shift in energy production is all around us.

Solar energy, in particular, has seen exponential growth. The cost of solar panels has plummeted, making it more accessible to homeowners and businesses. Innovative grid-tied systems allow excess energy to be sold back to the grid, creating an incentive for adoption. The efficiency of photovoltaic cells continues to improve, further driving the appeal of solar energy.

Wind energy is another success story. The sight of wind farms, with their large turbines harnessing the power of the wind, is becoming increasingly common. Advancements in turbine design and technology mean they can now generate power at lower wind speeds, increasing their utility and efficiency.

Hydropower, using the energy of flowing water to generate electricity, is not a new concept. However, recent technological innovations have revitalized this age-old method. New turbine designs, which are more fish-friendly and efficient, are being implemented in dams across the world.

But it's not just about technology. Governments and corporations are recognizing the value of renewable energy. Investments in green technologies are surging, and international agreements like the Paris Accord emphasize the global commitment to reducing carbon emissions.

Challenges remain, of course. The intermittent nature of solar and wind energy requires effective energy storage solutions. Battery technology, while advancing rapidly, still needs to evolve to meet the demands of a fully renewable-powered grid. There's also the challenge of overhauling existing infrastructure to accommodate these new energy sources.

However, the trajectory is clear. Renewable energy is no longer an alternative; it's the future. The combination of technological advancement, economic incentives, and environmental imperatives is pushing humanity towards a greener, more sustainable future.

""",1.5,'article')

Keywords are-: ['energy', 'wind', 'solar', 'renewable', 'grid']
Entities are-: ['the last decade', 'paris accord', 'hydropower']


"In the last decade, renewable energy has emerged from the fringes to become a mainstream powerhouse. Wind energy is another success story. Governments and corporations are recognizing the value of renewable energy. The intermittent nature of solar and wind energy requires effective energy storage solutions. Battery technology, while advancing rapidly, still needs to evolve to meet the demands of a fully renewable-powered grid. Renewable energy is no longer an alternative; it's the future."

In [212]:
dynamic_detailed_summarize_premium("""Developer community site Stack Overflow has laid off 28% of its staff, the Prosus-owned company announced Monday.

In a blog post, Stack Overflow’s CEO, Prashanth Chandrasekar indicated that the company is focusing on its path to profitability. While the post didn’t elaborate on the reason behind the job cuts, it mentioned customers’ budgets shifting elsewhere “due to the macroeconomic pressures.”

“This year we took many steps to spend less. Changes have been pursued through the lens of minimizing the impact on the lives of Stackers. Unfortunately, those changes were not enough and we have made the extremely difficult decision to reduce the company’s headcount by approximately 28%,” Chandrasekar said.

While Stack Overflow is primarily a Q&A website for consumers it also has enterprise products like “Stack Overflow for Teams,” which helps organizations maintain a company-wide knowledge base.

The company didn’t specify the number of laid-off employees. However, since it had pushed its headcount to over 500 people last year, more than 100 people are likely to be impacted.

With generative AI gaining popularity for helping coders with different problems, Stack Overflow has seen its traffic drop as compared to last year.

In August, the company said that because of generative AI, it expects “some rises and falls in traditional traffic and engagement over the coming months.”

Earlier this year, Stack Overflow asked AI companies to pay for training data. In January, it barred users from posting answers generated by AI. The company is also trying to bolster its own AI capabilities. In July, it launched OverflowAI with features like generative AI-powered search.

Big Tech is also moving fast to make generative AI-aided products available for coders in a rapid manner. Last month, GitHub expanded access to its Copilot chat to individual users. In May, during its developer conference, Google announced a bunch of AI-centric coding tools including an assistive bot called Codey. The company has also trained its conversational AI tool Bard to help users with code generation and debugging.""",1,'article')

Keywords are-: ['ai', 'company', 'overflow', 'stack', 'year']
Entities are-: ['over 500', 'stack overflow’s', 'july', 'ai', 'this year', 'the coming months', 'copilot', 'earlier this year', '28%', 'more than 100', 'stack overflow for teams', 'approximately 28%', 'january', 'codey', 'august', 'last year', 'github', 'prashanth chandrasekar', 'stack overflow', 'may', 'last month', 'chandrasekar', 'google', 'bard', 'monday', 'stackers']


'Developer community site Stack Overflow has laid off 28% of its staff, the Prosus-owned company announced Monday. While Stack Overflow is primarily a Q&A website for consumers it also has enterprise products like “Stack Overflow for Teams,” which helps organizations maintain a company-wide knowledge base. The company didn’t specify the number of laid-off employees. In May, during its developer conference, Google announced a bunch of AI-centric coding tools including an assistive bot called Codey.'

In [213]:
dynamic_detailed_summarize_premium("""Coffee is one of the most beloved beverages in the world, consumed by millions of people every day. The story of coffee begins in Ethiopia, where, according to legend, a goat herder named Kaldi discovered the coffee bean. Observing that his goats became more energetic after eating berries from a certain plant, Kaldi tried the berries himself and experienced a similar effect. He shared his discovery with a local monastery, where monks used the berries to make a drink that helped them stay awake during long hours of prayer.

From Ethiopia, coffee spread to the Arabian Peninsula. By the 15th century, it was being grown in Yemen, and from there, it made its way to other parts of the Middle East, North Africa, and eventually Europe. The drink became popular in Islamic countries as alcohol consumption was prohibited, and coffee provided a similar communal experience.

Coffee houses, or qahveh khaneh, began to appear in cities across the Near East. These establishments served as places where people could socialize, listen to music, play chess, and discuss current events. In the 17th century, coffee made its way to Europe, where it quickly became popular. Venice's first coffeehouse opened in 1645, and England's first coffeehouse opened in Oxford in 1650.

The demand for coffee beans grew, and European colonizers began to establish coffee plantations in various parts of the world. Java in Indonesia, Brazil, and the Caribbean became major centers for coffee production.

Today, coffee is grown in a multitude of countries around the world. Advances in technology have allowed for the development of various coffee-making methods, from the espresso machine to the French press to the drip brewer. There are also numerous coffee varieties and flavors, catering to a range of tastes.

In addition to its role as a popular beverage, coffee has played a significant role in many societies throughout history. It has been a source of inspiration for artists, writers, and thinkers. Coffee houses have served as centers of social activity, intellectual discussion, and even political plotting. The drink has also been the cause of controversy. At various points in history, it has been banned, taxed heavily, and sparked social and economic revolutions.

""",1)

Keywords are-: ['coffee', 'world', 'various', 'berries', 'drink']
Entities are-: ['first', 'kaldi', 'islamic', 'khaneh', 'yemen', 'england', 'french', '1650', 'caribbean', 'brazil', 'ethiopia', 'today', 'oxford', 'european', 'java', 'north africa', 'europe', 'indonesia', 'the arabian peninsula', '1645', 'the 15th century', 'every day', 'the near east', 'venice', 'long hours', 'the 17th century', 'millions', 'the middle east']


'Coffee is one of the most beloved beverages in the world, consumed by millions of people every day. The demand for coffee beans grew, and European colonizers began to establish coffee plantations in various parts of the world. Today, coffee is grown in a multitude of countries around the world.'

In [220]:
dynamic_detailed_summarize_premium("""Earlier this month, LinkedIn announced that it would roll out a raft of new AI-powered tools across the business. Today, it’s making a different kind of announcement focused on the future: The company is laying off 668 employees.

We understand from a reliable source that the majority of the layoffs, some 563, will be in R&D, with teams across engineering, product, talent and finance impacted.

The cuts, announced this morning, come five months after LinkedIn announced 716 job cuts, at the same time that it would be phasing out its app in China. Today’s cuts bring the total number of layoffs to Microsoft-owned LinkedIn to 1,384. In total, there have been more than 242,000 people laid off in the technology sector in 2023, according to employment tracker Layoffs.fyi.

“While we are adapting our organizational structures and streamlining our decision making, we are continuing to invest in strategic priorities for our future and to ensure we continue to deliver value for our members and customers,” the company said in an unsigned statement today. “We are committed to providing our full support to all impacted employees during this transition and ensuring that they are treated with care and respect.”

It didn’t specify which strategic priorities but a refocus on hiring more AI talent is likely part of the mix.

After getting acquired by Microsoft for more than $26 billion in 2016, LinkedIn became significantly less transparent in terms of its finances and other operational metrics. In Microsoft’s July 2023 full-fiscal-year earnings report, the company noted that it had more than 950 million members and over $15 billion in revenues, with Talent Solutions the single biggest contributor at $7 billion+.

“We continue to use AI to help our members and customers connect to opportunities and tap into the experiences of experts on the platform. Our AI-powered collaborative articles are now the fastest-growing traffic driver to LinkedIn,” it said at the time. """,1.5,'article')

Keywords are-: ['linkedin', 'ai', 'billion', 'announced', 'members']
Entities are-: ['this morning', '2016', 'over $15 billion', 'ai', '668', 'layoffs.fyi', 'fiscal-year', 'china', 'linkedin', 'today', 'more than 950 million', '2023', 'earlier this month', '7', 'talent solutions', 'microsoft', 'more than 242,000', 'more than $26 billion', 'some 563', '1,384', '716', 'july 2023', 'five months']


'Earlier this month, LinkedIn announced that it would roll out a raft of new AI-powered tools across the business. The cuts, announced this morning, come five months after LinkedIn announced 716 job cuts, at the same time that it would be phasing out its app in China. “We continue to use AI to help our members and customers connect to opportunities and tap into the experiences of experts on the platform. Our AI-powered collaborative articles are now the fastest-growing traffic driver to LinkedIn,” it said at the time.'

In [221]:
dynamic_detailed_summarize_premium("""The Impact of Digital Technology on Modern Businesses

Executive Summary:
Digital technology has drastically transformed the landscape of modern businesses. With the evolution of technologies like artificial intelligence, big data, and cloud computing, businesses are redefining their operational strategies and customer engagement models.

Introduction:
Over the past two decades, the rapid growth of digital technology has influenced every sector of the economy. From retail and healthcare to finance and manufacturing, the digital revolution is evident.

Digital Marketing and E-commerce:
The growth of digital platforms has given rise to new marketing strategies. Traditional advertising methods like television and print have been supplemented with digital marketing techniques such as search engine optimization, pay-per-click advertising, and social media marketing. E-commerce platforms like Amazon and Alibaba have transformed the retail landscape, allowing businesses to reach global audiences.

Data-Driven Decision Making:
Big data analytics enable businesses to gain insights from vast amounts of data. This has allowed for more informed decision-making processes and predictive analytics. Companies like Netflix and Spotify use big data to make content recommendations to their users, enhancing user engagement.

Operational Efficiency with AI and Automation:
Artificial intelligence and automation have streamlined business operations. Chatbots handle customer inquiries, while robotic process automation (RPA) systems manage repetitive tasks, increasing efficiency and reducing costs.

Remote Work and Cloud Computing:
The COVID-19 pandemic highlighted the importance of remote work capabilities. Cloud platforms like Microsoft Azure, Google Cloud, and AWS have enabled businesses to operate seamlessly in a remote environment.

Cybersecurity Concerns:
With the increased reliance on digital platforms, cybersecurity has become paramount. Businesses invest heavily in securing their data from cyber threats. Regular training and advanced cybersecurity measures are being adopted to protect sensitive information.

Conclusion:
The digital transformation journey has been beneficial for businesses, offering increased efficiency, wider customer reach, and innovative solutions. However, challenges like cybersecurity threats and the need for continuous upskilling of employees remain. As technology continues to evolve, businesses must stay adaptive and forward-thinking to harness its full potential.""",1,'report')

Keywords are-: ['digital', 'businesses', 'like', 'data', 'marketing']
Entities are-: ['digital technology', 'covid-19', 'alibaba', 'google cloud', 'digital', 'aws', 'spotify', 'netflix', 'rpa', 'remote work and cloud computing', 'microsoft azure', 'the past two decades', 'digital marketing', 'amazon']


'The Impact of Digital Technology on Modern Businesses\n\nExecutive Summary:\nDigital technology has drastically transformed the landscape of modern businesses. Introduction:\nOver the past two decades, the rapid growth of digital technology has influenced every sector of the economy. E-commerce platforms like Amazon and Alibaba have transformed the retail landscape, allowing businesses to reach global audiences.'

In [223]:
dynamic_detailed_summarize_premium1("""In a town in Persia, there lived two brothers — one named Cassim and the other Ali Baba. Cassim was rich, but Ali Baba was poor and earned his living by collecting wood in the forest. One day, while Ali Baba was working in the forest, he saw a group of 40 men on horseback. Curious, he climbed a tree to watch them.

The men stopped in front of a rock face and the leader shouted, "Open Sesame!" To Ali Baba's amazement, a door in the rock opened. The men entered, and the door shut behind them. After a while, the door opened again and the men left, the leader shouting, "Close Sesame!" to shut the door.

Once they were gone, Ali Baba approached the rock and used the magic words to open the door. Inside, he found a vast treasure. He took some gold coins and left, using the words to close the door behind him.

He told his brother Cassim about the treasure. Greedy Cassim went to the cave, took a lot of gold, but forgot the magic words to get out. The thieves found him, killed him, and hung his body at the entrance of the cave to scare off intruders.

Ali Baba found his brother's body, brought it home, and sought help from Morgiana, a clever slave-girl. Morgiana went to the thieves' den, learned of their plan to kill Ali Baba and cunningly killed them first. She poured boiling oil into jars where the thieves were hiding, killing them all.

Morgiana's bravery earned her freedom, and she married Ali Baba's son. Ali Baba, forever grateful, continued to visit the cave but ensured the secret remained safe, telling only those he trusted the most.

""",1.5,'story')

Keywords are-: ['ali', 'baba', 'door', 'men', 'cassim']
Entities are-: ['cassim', 'first', 'one', 'ali baba', 'two', 'morgiana', "ali baba's", 'open sesame', '40', 'persia', 'one day', 'greedy cassim']


"The men entered and the door shut behind them To Ali Baba 's amazement a door in the rock opened After a while the door opened and the men left the leader shouting Close Sesame He told his brother Cassim about the treasure Greedy Cassim went to the cave took a lot of gold but forgot the words to get out Morgiana went to the thieves ' den learned of their plan to kill Ali Baba and killed them Cassim was but Ali Baba was and earned his living by collecting wood in the forest One day while Ali Baba was working in the forest he saw a group of 40 men on horseback In a town in Persia there lived two brothers one named Cassim and the Ali Baba"

In [227]:
dynamic_detailed_summarize_premium("""AI adoption among Mac app users is booming, according to a new report from app subscription service Setapp that found that 42% of Mac users today report using AI-based apps on a daily basis, and 63% claim to believe that AI apps are more beneficial than those without AI. In addition, Mac app developers are embracing AI, with 44% having already implemented either AI or machine learning models in their apps while another 28% say they’re working on doing so.

The survey is part of an annual report on the state of Mac apps put out by the company, whose business involves a subscription service that provides access to more than 230 Mac apps. This year, its survey included responses from 1,241 Mac users, mostly in the U.S., so it does not necessarily provide insights into the adoption of AI-based apps on a more global basis. However, the report highlights the interest in AI apps among this portion of the Mac user base, noting that top AI apps included those that aren’t only native to macOS — like Google’s AI Bard and Bing, which integrated AI technology from OpenAI. In addition, other top AI apps users mentioned using include TypingMind, Elephas, Spark, Notion, Grammarly, Craft, Luminar Neo, MacGPT, Asana, Raycast and MacWhisper. Some of these leverage AI to augment their existing apps, as opposed to being an app that’s solely focused on interacting with AI.

“We see how AI is transforming the app’s usage experience by providing additional user assistance,” said Mykola Savin, product lead of Setapp, in an announcement. “At Setapp, we also witness a great adoption of AI tools and the features we implement on the platform. Maybe not everyone succeeds with AI on the first try. But when they do, they tend to use those features repeatedly.”

This is the first year Setapp has asked in its annual survey about AI app adoption, so it’s not possible to quantify how many more Mac users are now using AI apps daily, compared with years past. But AI apps’ slice of users’ daily workflows is significant, it seems. Out of an average of 51 installed apps on their Mac, users access up to 15 daily. And as 42% say they’re using AI apps daily, that means AI is now a large part of users’ daily workflows.

Other often-used apps include browser apps, Microsoft and Google Office tools (both of which are integrating AI technologies, as well), and Adobe software — the latter which has embraced generative AI across a range of apps, including Photoshop, and other Creative Cloud apps. So users’ true exposure to AI-powered apps may be even higher.

The survey also reports on other general findings about Mac app adoption and discovery, noting that subscriptions are more popular than one-time purchases, and the top means of app discovery include the Mac App store, YouTube and social media. In addition, 70% of the survey respondents report having a Mac with either an M1 or M2 chip, among other highlights. """,
2,'article')

Keywords are-: ['ai', 'apps', 'mac', 'app', 'users']
Entities are-: ['first', 'up to', 'setapp', 'adobe', 'm2', '1,241', 'ai', 'this year', '70%', 'asana', '42%', 'annual', 'spark', 'macwhisper', 'mykola savin', 'photoshop', 'mac', 'daily', 'macgpt', '51', '63%', 'openai', 'elephas', '28%', 'notion', 'craft', 'm1', 'today', 'the first year', '44%', 'years past', '15', 'youtube', 'google office', 'one', 'mac app', 'luminar neo', 'apps', 'u.s.', 'microsoft', 'google', 'bing', 'browser apps', 'creative cloud']


'AI adoption among Mac app users is booming, according to a new report from app subscription service Setapp that found that 42% of Mac users today report using AI-based apps on a daily basis, and 63% claim to believe that AI apps are more beneficial than those without AI. The survey is part of an annual report on the state of Mac apps put out by the company, whose business involves a subscription service that provides access to more than 230 Mac apps. This year, its survey included responses from 1,241 Mac users, mostly in the U.S., so it does not necessarily provide insights into the adoption of AI-based apps on a more global basis. However, the report highlights the interest in AI apps among this portion of the Mac user base, noting that top AI apps included those that aren’t only native to macOS — like Google’s AI Bard and Bing, which integrated AI technology from OpenAI. But when they do, they tend to use those features repeatedly.”\n\nThis is the first year Setapp has asked in its

In [229]:
dynamic_detailed_summarize_premium("""The Sam Bankman-Fried trial gained steam after a somewhat sleepier first half of the day. That’s when prosecutors and the defense asked a witness and former FTX developer about the technical details of the crypto exchange as well as Alameda Research.

But that changed around 4 p.m. when FTX co-founder and CTO Gary Wang took the stand, wearing a wrinkled suit. Prior to Wang taking the stand, there was a 15-minute break during which Bankman-Fried looked visibly irritated.

Bankman-Fried’s parents were also there. During the break, they went to their son seemingly in an effort to provide support. At one point his father, Joseph Bankman, patted his mother, Barbara Fried, on the back, said something and laughed. She didn’t laugh back but continued to look away toward her son.

On the stand, Wang admitted that he committed wire fraud, securities fraud and commodities fraud. He added that Bankman-Fried, Nishad Singh and Caroline Ellison were the individuals he committed the crimes with.

Wang, Singh and Ellison pleaded guilty in late December 2022 as part of a deal to cooperate with the government and testify during this trial.

Wang said that they were given “special privileges from Alameda Research,” the crypto trading firm that he said he and Bankman-Fried started prior to launching FTX. Those privileges included getting large lines of credit, unlimited withdrawals and being able to have negative balances. Wang said that the “unlimited funds” came from FTX customers; a special code was added to customer transactions that funneled the money to Alameda.

He shared during his testimony that he was in charge of writing and reviewing code. And while Bankman-Fried did not write the code, Wang said Bankman-Fried did tell him and other developers what to implement. “Sometimes we talked [disagreements] out, but in the end, it’s Sam’s decision,” Wang said.



Negative balances, unlimited withdrawals
Because of these special privileges, Alameda had a $65 billion line of credit, Wang said. “Normal large businesses have single to double digits [of credit] in the millions.” By the time the two businesses filed for bankruptcy in mid November 2022, Alameda withdrew $8 billion, Wang said.

These internal financial advantages were not disclosed to the public, he shared.

Alameda and FTX were both started by Bankman-Fried and Wang, with ownership split 90% and 10%, and then 17% equity and 65% equity, respectively. Singh also had 5% equity of FTX, and a number of outside investors held other positions, Wang noted.

The ownership percentages never changed, he added. At the time, both Wang and Bankman-Fried were billionaires.

During his time at the companies, Wang also disclosed that Alameda “loaned” him around $200 million to $300 million. But the money never went to his bank account, and it instead went to investments that FTX made into other companies.

Naming the business
The company also picked its name to outsmart other businesses, which might have negative connotations toward companies with crypto jargon in their titles. “Alameda” derived from Alameda County in California and “Research,” was because it “sounds prestigious” and is not using a crypto-related name, Wang said.

The initial funds for Alameda came from Bankman-Fried personally as well as various lenders. Wang said Bankman-Fried also believed it would be easier to get bank accounts, rental leases, investors and so on, with a more “normal” company name.

The prosecutors aired a clip of Bankman-Fried from April 2021 on a podcast, where he explained Alameda’s name. “If we named our company Shitcoin Day Trader’s Inc., no one would do business [with us],” he said at the time.

Wang’s testimony is expected to continue on Friday morning until midday, according to prosecutors at the trial.""",1,'article')

Keywords are-: ['wang', 'fried', 'bankman', 'said', 'alameda']
Entities are-: ['midday', 'first', 'shitcoin day trader’s inc.', 'barbara fried', 'bankman-fried', 'friday', 'joseph bankman', 'bankman-fried’s', 'nishad singh', 'april 2021', 'singh', 'california', 'around 4 p.m.', '15-minute', 'sam', 'sam bankman-fried', 'ftx', 'mid november 2022', 'alameda research', 'one', 'alameda', 'wang', 'two', '17%', '65%', 'late december 2022', '$8 billion', '90%', 'alameda county', 'cto', '10%', 'caroline ellison', 'gary wang', 'ellison', '$65 billion', 'around $200 million to $300 million', 'morning', 'millions', '5%']


'The Sam Bankman-Fried trial gained steam after a somewhat sleepier first half of the day. Bankman-Fried’s parents were also there. And while Bankman-Fried did not write the code, Wang said Bankman-Fried did tell him and other developers what to implement. “Sometimes we talked [disagreements] out, but in the end, it’s Sam’s decision,” Wang said. At the time, both Wang and Bankman-Fried were billionaires. “If we named our company Shitcoin Day Trader’s Inc., no one would do business [with us],” he said at the time.'